In [ ]:
# 检查 Arrow 数据集的列名
from datasets import load_from_disk

dataset_path = r"D:/桌面/DL/Pro/poetry_foundation"
dataset = load_from_disk(dataset_path)
train_dataset = dataset["train"]

print(train_dataset.column_names)  # 查看真实列名
print(train_dataset[0])  # 查看第一条记录


In [1]:
import json
from pathlib import Path

# SFT JSONL 文件路径
sft_path = Path("/root/autodl-tmp/Pro/poetry_sft.jsonl")

print("=== 前 15 条 SFT 样本 ===\n")

with open(sft_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 15:
            break
        record = json.loads(line.strip())
        messages = record.get("messages", [])
        
        print(f"--- SFT 样本 {i+1} ---")
        for msg in messages:
            role = msg.get("role")
            content = msg.get("content")
            print(f"{role}: {content}\n")
        print("------------------------------\n")


=== 前 15 条 SFT 样本 ===

--- SFT 样本 1 ---
user: Write a poem in the style of Michelle Menting.

assistant: Dog bone, stapler,
cribbage board, garlic press
     because this window is loose—lacks
suction, lacks grip.
Bungee cord, bootstrap,
dog leash, leather belt
     because this window had sash cords.
They frayed. They broke.
Feather duster, thatch of straw, empty
bottle of Elmer's glue
     because this window is loud—its hinges clack
open, clack shut.
Stuffed bear, baby blanket,
single crib newel
     because this window is split. It's dividing
in two.
Velvet moss, sagebrush,
willow branch, robin's wing
     because this window, it's pane-less. It's only
a frame of air.

------------------------------

--- SFT 样本 2 ---
user: Write a poem in the style of Lucia Cherciu.

assistant: The old cupola glinted above the clouds, shone
among fir trees, but it took him an hour
for the half mile all the way up the hill. As he trailed,
the village passed him by, greeted him,
asked about his healt

In [1]:
import json
import torch
from pathlib import Path
from tokenizers import Tokenizer

# ---------------------------
# 本地 tokenizer
# ---------------------------
tokenizer_path = r"/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b/tokenizer.json"
tokenizer = Tokenizer.from_file(tokenizer_path)

# ---------------------------
# 读取 JSONL 的第一条样本
# ---------------------------
sft_path = Path(r"/root/autodl-tmp/Pro/poetry_sft.jsonl")
with open(sft_path, "r", encoding="utf-8") as f:
    first_line = f.readline()
    sample_jsonl = json.loads(first_line.strip())

# ---------------------------
# 拼接 user 和 assistant
# ---------------------------
user_text = ""
assistant_text = ""

for msg in sample_jsonl.get("messages", []):
    if msg["role"] == "user":
        user_text += msg["content"].strip() + "\n"
    elif msg["role"] == "assistant":
        assistant_text += msg["content"].strip() + "\n"

full_text = user_text + assistant_text

# ---------------------------
# Tokenization
# ---------------------------
input_ids = tokenizer.encode(full_text).ids
user_len = len(tokenizer.encode(user_text).ids)

input_ids = torch.tensor(input_ids).unsqueeze(0)  # batch=1
labels = input_ids.clone()
labels[:, :user_len] = -100  # user 部分不计算 loss

# ---------------------------
# 假设你有一个本地 PyTorch LLM 模型（Qwen2.5-7b）
# model.forward 返回 logits (batch, seq_len, vocab_size)
# 这里演示如何计算 assistant loss
# ---------------------------

# logits 这里用随机张量模拟，实际用你本地模型前向
batch, seq_len = input_ids.shape
vocab_size = tokenizer.get_vocab_size()
logits = torch.randn(batch, seq_len, vocab_size)  # 模拟模型输出

# CrossEntropyLoss 计算
loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
loss = loss_fct(logits.view(-1, vocab_size), labels.view(-1))

print("=== 第一条样本 assistant loss ===")
print(loss.item())


=== 第一条样本 assistant loss ===
12.385249137878418


In [22]:
# -------------------------
# 2. 加载 tokenizer（方案 B，本地可用）
# -------------------------
tokenizer_file = os.path.join(CONFIG["model_name_or_path"], "tokenizer.json")
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)

# 手动设置
tokenizer.pad_token = "<|pad|>"  # 填充序列
tokenizer.eos_token = "<|endoftext|>"  # 文本结束
tokenizer.bos_token = "<|startoftext|>"  # 文本开始

# 手动设置特殊 token
if tokenizer.pad_token is None:
    tokenizer.pad_token = "<|pad|>"
if tokenizer.eos_token is None:
    tokenizer.eos_token = "<|endoftext|>"
if tokenizer.bos_token is None:
    tokenizer.bos_token = "<|startoftext|>"

print("pad_token:", tokenizer.pad_token)
print("eos_token:", tokenizer.eos_token)
print("bos_token:", tokenizer.bos_token)
print("词表大小:", len(tokenizer))


pad_token: <|pad|>
eos_token: <|endoftext|>
bos_token: <|startoftext|>
词表大小: 151665


In [ ]:
# %%
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import torch.nn as nn
import os
import time
from tqdm import tqdm  # 进度条

# -------------------------
# 0. 全局缓存目录到数据盘
# -------------------------
os.environ["TRANSFORMERS_CACHE"] = "/root/autodl-tmp/hf_cache"
torch.hub.set_dir("/root/autodl-tmp/torch_cache")
os.environ["TORCH_HOME"] = "/root/autodl-tmp/torch_cache"
os.makedirs("/root/autodl-tmp/offload", exist_ok=True)
os.makedirs("/root/autodl-tmp/hf_cache", exist_ok=True)
os.makedirs("/root/autodl-tmp/torch_cache", exist_ok=True)

# -------------------------
# 1. 配置
# -------------------------
CONFIG = {
    "model_name_or_path": "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b",
    "data_path": "/root/autodl-tmp/Pro/poetry_sft.jsonl",
    "output_dir": "/root/autodl-tmp/Pro/qwen2.5-sft",
    "num_train_epochs": 3,
    "batch_size": 1,
    "gradient_accumulation_steps": 4,
    "learning_rate": 2e-5,
    "max_seq_length": 512,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "offload_folder": "/root/autodl-tmp/offload",
}

# -------------------------
# 2. 加载 tokenizer
# -------------------------
tokenizer_file = os.path.join(CONFIG["model_name_or_path"], "tokenizer.json")
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
tokenizer.eos_token = "<|endoftext|>"
tokenizer.bos_token = "<|startoftext|>"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------------
# 3. Dataset
# -------------------------
class PoetryDataset(Dataset):
    def __init__(self, jsonl_path, tokenizer, max_len=512):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        user_text = sample["messages"][0]["content"]
        assistant_text = sample["messages"][1]["content"]

        prompt = f"<|im_start|>user\n{user_text}<|im_end|>\n<|im_start|>assistant\n{assistant_text}<|im_end|>"
        input_ids = self.tokenizer(
            prompt, max_length=self.max_len, padding="max_length", truncation=True
        )["input_ids"]

        user_prompt_ids = self.tokenizer(
            f"<|im_start|>user\n{user_text}<|im_end|>\n", truncation=True, max_length=self.max_len
        )["input_ids"]

        labels = [-100]*len(input_ids)
        for i, tok in enumerate(input_ids[len(user_prompt_ids):]):
            labels[len(user_prompt_ids)+i] = tok

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long),
            "attention_mask": torch.tensor(
                [1 if id != self.tokenizer.pad_token_id else 0 for id in input_ids],
                dtype=torch.long
            )
        }

dataset = PoetryDataset(CONFIG["data_path"], tokenizer, CONFIG["max_seq_length"])
loader = DataLoader(dataset, batch_size=CONFIG["batch_size"], shuffle=True)

# -------------------------
# 4. 加载模型 + LoRA
# -------------------------
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["model_name_or_path"],
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder=CONFIG["offload_folder"]
)
model.gradient_checkpointing_enable()
model.train()

lora_cfg = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj","v_proj","k_proj","o_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

# -------------------------
# 5. Optimizer + Loss
# -------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"])
loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

# -------------------------
# 6. 训练循环（实时显示每个 sample loss + 剩余时间）
# -------------------------
for epoch in range(CONFIG["num_train_epochs"]):
    total_loss = 0.0
    start_time = time.time()  # ===== 新增：记录epoch开始时间 =====
    for step, batch in enumerate(loader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(CONFIG["device"])
        labels = batch["labels"].to(CONFIG["device"])
        attention_mask = batch["attention_mask"].to(CONFIG["device"])

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # ===== 新增：计算时间相关 =====
        elapsed_time = time.time() - start_time  # 已用时间（秒）
        steps_done = step + 1
        steps_total = len(loader)
        time_per_step = elapsed_time / steps_done  # 每步耗时
        remaining_steps = steps_total - steps_done  # 剩余步数
        remaining_time = remaining_steps * time_per_step  # 剩余时间（秒）
        
        # 格式化为 时:分:秒
        def format_time(seconds):
            hours = int(seconds // 3600)
            minutes = int((seconds % 3600) // 60)
            secs = int(seconds % 60)
            return f"{hours:02d}:{minutes:02d}:{secs:02d}"
        
        elapsed_str = format_time(elapsed_time)
        remaining_str = format_time(remaining_time)

        # ===== 修改打印行：添加时间显示 =====
        print(f"\rEpoch {epoch+1} | Step {step+1}/{len(loader)} | current_loss: {loss.item():.4f} | 已用: {elapsed_str} | 剩余: {remaining_str}", end="")

    # 每个 epoch 结束换行，并打印平均 loss
    total_epoch_time = format_time(time.time() - start_time)
    print(f"\nEpoch {epoch+1} 平均 loss: {total_loss/len(loader):.4f} | 总耗时: {total_epoch_time}")

# -------------------------
# 7. 保存 LoRA 模型
# -------------------------
model.save_pretrained(CONFIG["output_dir"])
tokenizer.save_pretrained(CONFIG["output_dir"])
print("🎉 LoRA 微调完成！")

In [ ]:
# %%
import json
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import torch.nn as nn
import os
import time
import random
import nltk
from nltk.corpus import wordnet
from tqdm import tqdm  # 进度条

# -------------------------
# 0. 全局缓存目录到数据盘 + NLTK资源下载
# -------------------------
# 配置缓存路径
os.environ["TRANSFORMERS_CACHE"] = "/root/autodl-tmp/hf_cache"
os.environ["HF_HOME"] = "/root/autodl-tmp/hf_cache"
torch.hub.set_dir("/root/autodl-tmp/torch_cache")
os.environ["TORCH_HOME"] = "/root/autodl-tmp/torch_cache"
# 创建目录
os.makedirs("/root/autodl-tmp/offload", exist_ok=True)
os.makedirs("/root/autodl-tmp/hf_cache", exist_ok=True)
os.makedirs("/root/autodl-tmp/torch_cache", exist_ok=True)

# 下载NLTK英文同义词库（首次运行自动下载）
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')

# -------------------------
# 1. 配置（新增创造度相关参数）
# -------------------------
CONFIG = {
    "model_name_or_path": "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b",
    "data_path": "/root/autodl-tmp/Pro/poetry_sft.jsonl",  # 英文诗JSONL数据
    "output_dir": "/root/autodl-tmp/Pro/qwen2.5-sft",
    "num_train_epochs": 3,
    "batch_size": 1,
    "gradient_accumulation_steps": 4,  # 梯度累积（已启用）
    "learning_rate": 2e-5,
    "max_seq_length": 512,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "offload_folder": "/root/autodl-tmp/offload",
    # 创造度提升参数
    "diversity_alpha": 0.1,  # 多样性损失权重（0.05~0.2）
    "temperature": 1.2,      # 推理时温度系数（越高创造度越强）
    "top_k": 50,             # 推理采样top_k
    "top_p": 0.95,           # 推理采样top_p
    "repetition_penalty": 1.1 # 重复惩罚（避免诗句重复）
}

# -------------------------
# 2. 加载 tokenizer（适配英文）
# -------------------------
tokenizer_file = os.path.join(CONFIG["model_name_or_path"], "tokenizer.json")
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
tokenizer.eos_token = "<|endoftext|>"
tokenizer.bos_token = "<|startoftext|>"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------------
# 3. Dataset（英文诗+文本增强）
# -------------------------
class PoetryDataset(Dataset):
    def __init__(self, jsonl_path, tokenizer, max_len=512):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        # 加载英文诗数据
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    # 英文文本增强：同义词替换（提升训练数据多样性）
    def augment_english_text(self, text):
        tokens = text.split()
        if len(tokens) < 3:
            return text
        
        # 词性标注，仅替换名词/动词/形容词
        pos_tags = nltk.pos_tag(tokens)
        augmented_tokens = []
        for token, pos in pos_tags:
            if pos.startswith('N') or pos.startswith('V') or pos.startswith('J'):
                # 从WordNet获取同义词
                synonyms = wordnet.synsets(token)
                if synonyms:
                    synonym_lemma = synonyms[0].lemmas()[0].name()
                    # 保持诗歌韵律，避免过长替换
                    if synonym_lemma != token and len(synonym_lemma) <= len(token)+2:
                        augmented_tokens.append(synonym_lemma.replace('_', ' '))
                    else:
                        augmented_tokens.append(token)
                else:
                    augmented_tokens.append(token)
            else:
                augmented_tokens.append(token)
        return ' '.join(augmented_tokens)

    def __getitem__(self, idx):
        sample = self.data[idx]
        user_text = sample["messages"][0]["content"]  # 英文主题
        assistant_text = sample["messages"][1]["content"]  # 英文诗

        # 50%概率增强诗歌文本（提升创造度）
        if random.random() < 0.5:
            assistant_text = self.augment_english_text(assistant_text)

        # 构建英文prompt
        prompt = f"<|im_start|>user\n{user_text}<|im_end|>\n<|im_start|>assistant\n{assistant_text}<|im_end|>"
        input_ids = self.tokenizer(
            prompt, max_length=self.max_len, padding="max_length", truncation=True
        )["input_ids"]

        # 分离user和assistant部分，仅计算assistant的loss
        user_prompt_ids = self.tokenizer(
            f"<|im_start|>user\n{user_text}<|im_end|>\n", truncation=True, max_length=self.max_len
        )["input_ids"]

        # 标签：user部分设为-100（忽略），assistant部分为真实token
        labels = [-100]*len(input_ids)
        for i, tok in enumerate(input_ids[len(user_prompt_ids):]):
            labels[len(user_prompt_ids)+i] = tok

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long),
            "attention_mask": torch.tensor(
                [1 if id != self.tokenizer.pad_token_id else 0 for id in input_ids],
                dtype=torch.long
            )
        }

dataset = PoetryDataset(CONFIG["data_path"], tokenizer, CONFIG["max_seq_length"])
loader = DataLoader(dataset, batch_size=CONFIG["batch_size"], shuffle=True)

# -------------------------
# 4. 加载模型 + LoRA
# -------------------------
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["model_name_or_path"],
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder=CONFIG["offload_folder"]
)
model.gradient_checkpointing_enable()
model.train()

# LoRA配置（适配英文诗生成）
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj","v_proj","k_proj","o_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

# -------------------------
# 5. 损失函数（新增多样性损失）
# -------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"])
ce_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

# 多样性损失：惩罚保守预测，提升创造度
def diversity_loss(logits, alpha=CONFIG["diversity_alpha"]):
    probs = F.softmax(logits, dim=-1)  # [batch*seq_len, vocab_size]
    # 计算熵值：熵越高，预测分布越分散（创造度越高）
    entropy = -torch.sum(probs * torch.log(probs + 1e-8), dim=-1)
    # 归一化熵值到0-1区间
    entropy_norm = entropy / torch.log(torch.tensor(probs.size(-1), device=logits.device))
    # 惩罚低熵值（保守预测）
    penalty = alpha * (1 - entropy_norm)
    return torch.mean(penalty)

# -------------------------
# 6. 训练循环（梯度累积+多损失+创造度优化）
# -------------------------
# 时间格式化函数
def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

# 启用梯度累积的训练循环
for epoch in range(CONFIG["num_train_epochs"]):
    total_loss = 0.0
    total_ce_loss = 0.0
    total_div_loss = 0.0
    start_time = time.time()
    optimizer.zero_grad()  # 初始化梯度
    
    for step, batch in enumerate(loader):
        # 加载数据到设备
        input_ids = batch["input_ids"].to(CONFIG["device"])
        labels = batch["labels"].to(CONFIG["device"])
        attention_mask = batch["attention_mask"].to(CONFIG["device"])

        # 前向传播
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # 计算损失
        ce_loss = ce_loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))  # 基础交叉熵
        div_loss = diversity_loss(logits.view(-1, logits.size(-1)))            # 多样性损失
        total_step_loss = (ce_loss + div_loss) / CONFIG["gradient_accumulation_steps"]  # 梯度累积均分

        # 反向传播（累积梯度）
        total_step_loss.backward()

        # 每gradient_accumulation_steps步更新一次参数
        if (step + 1) % CONFIG["gradient_accumulation_steps"] == 0:
            optimizer.step()
            optimizer.zero_grad()

        # 累计损失
        total_loss += total_step_loss.item() * CONFIG["gradient_accumulation_steps"]
        total_ce_loss += ce_loss.item()
        total_div_loss += div_loss.item()

        # 实时打印进度（含创造度相关指标）
        elapsed_time = time.time() - start_time
        steps_done = step + 1
        steps_total = len(loader)
        time_per_step = elapsed_time / steps_done
        remaining_steps = steps_total - steps_done
        remaining_time = remaining_steps * time_per_step

        print(f"\rEpoch {epoch+1} | Step {steps_done}/{steps_total} | Total Loss: {total_step_loss.item()*CONFIG['gradient_accumulation_steps']:.4f} | "
              f"CE Loss: {ce_loss.item():.4f} | Div Loss: {div_loss.item():.4f} | "
              f"已用: {format_time(elapsed_time)} | 剩余: {format_time(remaining_time)}", end="")

    # Epoch结束打印总结
    epoch_avg_loss = total_loss / len(loader)
    epoch_avg_ce = total_ce_loss / len(loader)
    epoch_avg_div = total_div_loss / len(loader)
    total_epoch_time = format_time(time.time() - start_time)
    print(f"\nEpoch {epoch+1} 完成 | 平均总Loss: {epoch_avg_loss:.4f} | 平均CE Loss: {epoch_avg_ce:.4f} | 平均Div Loss: {epoch_avg_div:.4f} | 总耗时: {total_epoch_time}\n")

# -------------------------
# 7. 保存 LoRA 模型
# -------------------------
model.save_pretrained(CONFIG["output_dir"])
tokenizer.save_pretrained(CONFIG["output_dir"])
print("🎉 LoRA 微调完成！模型保存至:", CONFIG["output_dir"])

# -------------------------
# 8. 高创造度英文诗生成函数
# -------------------------
def generate_creative_poetry(topic, max_len=512):
    """生成高创造度的英文诗"""
    model.eval()
    # 构建英文prompt
    prompt = f"<|im_start|>user\n{topic}<|im_end|>\n<|im_start|>assistant\n"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(CONFIG["device"])
    
    # 高创造度采样策略
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_length=max_len,
            do_sample=True,               # 启用采样（核心：提升创造度）
            top_k=CONFIG["top_k"],        # 仅从概率前K的token中选
            top_p=CONFIG["top_p"],        # 累积概率采样
            temperature=CONFIG["temperature"],  # 温度系数（越高越多样）
            repetition_penalty=CONFIG["repetition_penalty"],  # 惩罚重复
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # 解码并清理生成结果
    poetry = tokenizer.decode(outputs[0], skip_special_tokens=True)
    poetry = poetry.replace(prompt, "").strip()
    return poetry

# -------------------------
# 9. 生成示例
# -------------------------
if __name__ == "__main__":
    # 生成关于"starry night"的高创造度英文诗
    print("\n=== 生成高创造度英文诗 ===")
    topic = "starry night"
    poetry = generate_creative_poetry(topic)
    print(f"主题: {topic}\n")
    print(poetry)

In [6]:
# %%
import json
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import torch.nn as nn
import os
import time
import random
import nltk
from nltk.corpus import wordnet
from tqdm import tqdm  # 进度条

# -------------------------
# 0. 全局缓存目录到数据盘 + NLTK资源下载
# -------------------------
# 配置缓存路径
os.environ["TRANSFORMERS_CACHE"] = "/root/autodl-tmp/hf_cache"
os.environ["HF_HOME"] = "/root/autodl-tmp/hf_cache"
torch.hub.set_dir("/root/autodl-tmp/torch_cache")
os.environ["TORCH_HOME"] = "/root/autodl-tmp/torch_cache"
# 创建目录
os.makedirs("/root/autodl-tmp/offload", exist_ok=True)
os.makedirs("/root/autodl-tmp/hf_cache", exist_ok=True)
os.makedirs("/root/autodl-tmp/torch_cache", exist_ok=True)

# 下载NLTK英文同义词库（首次运行自动下载）
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')

# -------------------------
# 1. 配置（新增创造度相关参数 + 数值稳定参数）
# -------------------------
CONFIG = {
    "model_name_or_path": "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b",
    "data_path": "/root/autodl-tmp/Pro/poetry_sft.jsonl",  # 英文诗JSONL数据
    "output_dir": "/root/autodl-tmp/Pro/qwen2.5-sft",
    "num_train_epochs": 3,
    "batch_size": 1,
    "gradient_accumulation_steps": 4,  # 梯度累积
    "learning_rate": 2e-5,
    "max_seq_length": 512,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "offload_folder": "/root/autodl-tmp/offload",
    # 创造度提升参数
    "diversity_alpha": 0.05,  # 降低权重，先从0.05开始（原0.1）
    "temperature": 1.2,      
    "top_k": 50,             
    "top_p": 0.95,           
    "repetition_penalty": 1.1,
    # 数值稳定参数
    "eps": 1e-6,  # 防止除0/对数溢出
    "max_norm": 1.0,  # 梯度裁剪
    "fp32_loss": True  # 损失计算用fp32，避免fp16溢出
}

# -------------------------
# 2. 加载 tokenizer（适配英文）
# -------------------------
tokenizer_file = os.path.join(CONFIG["model_name_or_path"], "tokenizer.json")
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
tokenizer.eos_token = "<|endoftext|>"
tokenizer.bos_token = "<|startoftext|>"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------------
# 3. Dataset（核心修复：确保__len__有效 + 数据加载校验）
# -------------------------
class PoetryDataset(Dataset):
    def __init__(self, jsonl_path, tokenizer, max_len=512):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.data = self._load_and_validate_data(jsonl_path)  # 独立加载函数
        
        # 关键校验：确保数据非空
        if len(self.data) == 0:
            raise ValueError(f"数据加载失败！路径 {jsonl_path} 下无有效数据，请检查文件格式和路径")
        print(f"✅ 成功加载有效数据条数: {len(self.data)}")

    def _load_and_validate_data(self, jsonl_path):
        """独立的数据加载+校验函数，确保返回非空列表"""
        valid_data = []
        
        # 第一步：校验文件是否存在
        if not os.path.exists(jsonl_path):
            raise FileNotFoundError(f"数据文件不存在: {jsonl_path}")
        
        # 第二步：加载并过滤有效数据
        with open(jsonl_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            if len(lines) == 0:
                raise ValueError(f"数据文件为空: {jsonl_path}")
            
            for line_idx, line in enumerate(lines):
                line = line.strip()
                if not line:  # 跳过空行
                    continue
                try:
                    sample = json.loads(line)
                    # 校验messages结构
                    if not isinstance(sample.get("messages"), list) or len(sample["messages"]) < 2:
                        print(f"⚠️  跳过第{line_idx+1}行：messages格式错误")
                        continue
                    # 校验user/assistant内容
                    user_msg = sample["messages"][0]
                    assistant_msg = sample["messages"][1]
                    if (not isinstance(user_msg.get("content"), str) or not user_msg["content"].strip() or
                        not isinstance(assistant_msg.get("content"), str) or not assistant_msg["content"].strip()):
                        print(f"⚠️  跳过第{line_idx+1}行：内容为空或格式错误")
                        continue
                    # 所有校验通过，加入有效数据
                    valid_data.append(sample)
                except json.JSONDecodeError:
                    print(f"⚠️  跳过第{line_idx+1}行：JSON解析失败")
                except Exception as e:
                    print(f"⚠️  跳过第{line_idx+1}行：未知错误 - {str(e)}")
        
        return valid_data

    # 修复英文文本增强：增加边界校验
    def augment_english_text(self, text):
        if not text or len(text.split()) < 3:
            return text
        
        try:
            # 词性标注，仅替换名词/动词/形容词
            pos_tags = nltk.pos_tag(text.split())
            augmented_tokens = []
            for token, pos in pos_tags:
                if pos.startswith('N') or pos.startswith('V') or pos.startswith('J'):
                    synonyms = wordnet.synsets(token)
                    if synonyms and len(synonyms) > 0:
                        synonym_lemma = synonyms[0].lemmas()[0].name()
                        # 严格校验替换词
                        if (synonym_lemma != token and 
                            len(synonym_lemma) <= len(token)+2 and
                            not synonym_lemma.isdigit()):
                            augmented_tokens.append(synonym_lemma.replace('_', ' '))
                        else:
                            augmented_tokens.append(token)
                    else:
                        augmented_tokens.append(token)
                else:
                    augmented_tokens.append(token)
            return ' '.join(augmented_tokens)
        except:
            return text  # 增强失败时返回原文本

    def __len__(self):
        """核心修复：确保返回非负整数，且数据加载时已校验非空"""
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        user_text = sample["messages"][0]["content"].strip()
        assistant_text = sample["messages"][1]["content"].strip()

        # 50%概率增强诗歌文本（提升创造度）
        if random.random() < 0.5:
            assistant_text = self.augment_english_text(assistant_text)

        # 构建英文prompt
        prompt = f"<|im_start|>user\n{user_text}<|im_end|>\n<|im_start|>assistant\n{assistant_text}<|im_end|>"
        # 编码（增加padding_side='right'，避免截断assistant部分）
        encoding = self.tokenizer(
            prompt, 
            max_length=self.max_len, 
            padding="max_length", 
            truncation=True,
            padding_side='right'
        )
        input_ids = encoding["input_ids"]
        attention_mask = encoding["attention_mask"]

        # 分离user和assistant部分，仅计算assistant的loss
        user_prompt = f"<|im_start|>user\n{user_text}<|im_end|>\n<|im_start|>assistant\n"
        user_prompt_ids = self.tokenizer(
            user_prompt, 
            truncation=True, 
            max_length=self.max_len
        )["input_ids"]
        user_prompt_len = min(len(user_prompt_ids), self.max_len-1)  # 防止越界

        # 标签：user部分设为-100（忽略），assistant部分为真实token
        labels = [-100]*len(input_ids)
        # 确保不越界
        for i in range(user_prompt_len, min(len(input_ids), self.max_len)):
            labels[i] = input_ids[i]

        # 校验：确保有非-100的标签（避免CE Loss为0）
        if sum(1 for x in labels if x != -100) < 5:
            # 替换为简单标签（防止nan）
            labels = input_ids.copy()

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long)
        }

# -------------------------
# 4. 数据加载（增加异常捕获）
# -------------------------
try:
    dataset = PoetryDataset(CONFIG["data_path"], tokenizer, CONFIG["max_seq_length"])
    # 修复DataLoader：明确指定sampler的num_samples（防止空数据）
    loader = DataLoader(
        dataset, 
        batch_size=CONFIG["batch_size"], 
        shuffle=True,
        num_workers=0,  # 禁用多进程，避免数据加载异常
        drop_last=True  # 丢弃最后不完整的batch
    )
except Exception as e:
    print(f"❌ 数据加载失败: {str(e)}")
    exit(1)

# -------------------------
# 5. 加载模型 + LoRA
# -------------------------
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["model_name_or_path"],
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder=CONFIG["offload_folder"],
    # 增加数值稳定配置
    attn_implementation="eager" # 禁用flash attention（可能导致fp16溢出）
)
model.gradient_checkpointing_enable()
model.train()

# LoRA配置（适配英文诗生成）
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj","v_proj","k_proj","o_proj"],
    task_type="CAUSAL_LM",
    bias="none"  # 禁用bias训练，减少数值波动
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

# -------------------------
# 6. 损失函数（修复多样性损失 + 数值稳定）
# -------------------------
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=CONFIG["learning_rate"],
    eps=1e-8,  # 优化器数值稳定
    weight_decay=0.01  # 增加权重衰减，防止过拟合
)
ce_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

# 修复多样性损失：增加数值稳定处理
def diversity_loss(logits, alpha=CONFIG["diversity_alpha"]):
    # 转换为fp32计算，避免fp16溢出
    if CONFIG["fp32_loss"]:
        logits = logits.float()
    
    # 计算softmax（增加dim=-1，防止维度错误）
    probs = F.softmax(logits, dim=-1)
    # 限制probs的范围，防止log(0)
    probs = torch.clamp(probs, min=CONFIG["eps"], max=1.0 - CONFIG["eps"])
    
    # 计算熵值（数值稳定版）
    entropy = -torch.sum(probs * torch.log(probs), dim=-1)
    # 计算最大熵（vocab_size的对数）
    vocab_size = probs.size(-1)
    max_entropy = torch.log(torch.tensor(vocab_size, dtype=torch.float32, device=logits.device))
    # 归一化熵值（防止除0）
    entropy_norm = entropy / (max_entropy + CONFIG["eps"])
    # 限制熵值范围在0-1
    entropy_norm = torch.clamp(entropy_norm, min=0.0, max=1.0)
    
    # 惩罚低熵值（保守预测）
    penalty = alpha * (1 - entropy_norm)
    # 防止penalty出现nan
    penalty = torch.nan_to_num(penalty, nan=0.0, posinf=alpha, neginf=0.0)
    
    return torch.mean(penalty)

# -------------------------
# 7. 训练循环（梯度累积+多损失+创造度优化+数值稳定）
# -------------------------
# 时间格式化函数
def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

# 启用梯度累积的训练循环
for epoch in range(CONFIG["num_train_epochs"]):
    total_loss = 0.0
    total_ce_loss = 0.0
    total_div_loss = 0.0
    start_time = time.time()
    optimizer.zero_grad()  # 初始化梯度
    
    # 增加空loader校验
    if len(loader) == 0:
        print("❌ 训练加载器为空，终止训练")
        break
    
    for step, batch in enumerate(loader):
        # 加载数据到设备
        input_ids = batch["input_ids"].to(CONFIG["device"])
        labels = batch["labels"].to(CONFIG["device"])
        attention_mask = batch["attention_mask"].to(CONFIG["device"])

        # 前向传播
        outputs = model(
            input_ids=input_ids, 
            attention_mask=attention_mask,
            labels=labels  # 让模型自动计算loss（双重校验）
        )
        logits = outputs.logits
        # 基础loss（双重校验）
        base_loss = outputs.loss
        
        # 计算损失（数值稳定版）
        # 方式1：用模型自带的loss（更稳定）
        ce_loss = base_loss.float() if CONFIG["fp32_loss"] else base_loss
        # 方式2：手动计算CE Loss（备用）
        # ce_loss = ce_loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
        # if CONFIG["fp32_loss"]:
        #     ce_loss = ce_loss.float()
        
        # 计算多样性损失
        div_loss = diversity_loss(logits.view(-1, logits.size(-1)))
        # 总损失
        total_step_loss = (ce_loss + div_loss) / CONFIG["gradient_accumulation_steps"]

        # 检查loss是否为nan
        if torch.isnan(total_step_loss):
            print(f"\n⚠️  Step {step} 出现NaN Loss，跳过该步")
            optimizer.zero_grad()
            continue

        # 反向传播（累积梯度）
        total_step_loss.backward()

        # 梯度裁剪（关键：防止梯度爆炸）
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=CONFIG["max_norm"])

        # 每gradient_accumulation_steps步更新一次参数
        if (step + 1) % CONFIG["gradient_accumulation_steps"] == 0:
            optimizer.step()
            optimizer.zero_grad()

        # 累计损失（转换为float，防止fp16溢出）
        total_loss += total_step_loss.item() * CONFIG["gradient_accumulation_steps"]
        total_ce_loss += ce_loss.item()
        total_div_loss += div_loss.item()

        # 实时打印进度（含创造度相关指标）
        elapsed_time = time.time() - start_time
        steps_done = step + 1
        steps_total = len(loader)
        time_per_step = elapsed_time / steps_done
        remaining_steps = steps_total - steps_done
        remaining_time = remaining_steps * time_per_step

        print(f"\rEpoch {epoch+1} | Step {steps_done}/{steps_total} | Total Loss: {total_step_loss.item()*CONFIG['gradient_accumulation_steps']:.4f} | "
              f"CE Loss: {ce_loss.item():.4f} | Div Loss: {div_loss.item():.4f} | "
              f"已用: {format_time(elapsed_time)} | 剩余: {format_time(remaining_time)}", end="")

    # Epoch结束打印总结
    epoch_avg_loss = total_loss / len(loader)
    epoch_avg_ce = total_ce_loss / len(loader)
    epoch_avg_div = total_div_loss / len(loader)
    total_epoch_time = format_time(time.time() - start_time)
    print(f"\nEpoch {epoch+1} 完成 | 平均总Loss: {epoch_avg_loss:.4f} | 平均CE Loss: {epoch_avg_ce:.4f} | 平均Div Loss: {epoch_avg_div:.4f} | 总耗时: {total_epoch_time}\n")

# -------------------------
# 8. 保存 LoRA 模型
# -------------------------
model.save_pretrained(CONFIG["output_dir"])
tokenizer.save_pretrained(CONFIG["output_dir"])
print("🎉 LoRA 微调完成！模型保存至:", CONFIG["output_dir"])

# -------------------------
# 9. 高创造度英文诗生成函数
# -------------------------
def generate_creative_poetry(topic, max_len=512):
    """生成高创造度的英文诗"""
    model.eval()
    # 构建英文prompt
    prompt = f"<|im_start|>user\n{topic}<|im_end|>\n<|im_start|>assistant\n"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(CONFIG["device"])
    
    # 高创造度采样策略
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_length=max_len,
            do_sample=True,               
            top_k=CONFIG["top_k"],        
            top_p=CONFIG["top_p"],        
            temperature=CONFIG["temperature"],  
            repetition_penalty=CONFIG["repetition_penalty"],  
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            # 生成时数值稳定配置
            num_return_sequences=1,
            no_repeat_ngram_size=2,  # 避免2元语法重复
            clean_up_tokenization_spaces=True
        )
    
    # 解码并清理生成结果
    poetry = tokenizer.decode(outputs[0], skip_special_tokens=True)
    poetry = poetry.replace(prompt.replace("<|im_start|>", "").replace("<|im_end|>", "").strip(), "").strip()
    return poetry

# -------------------------
# 10. 生成示例
# -------------------------
if __name__ == "__main__":
    # 生成关于"starry night"的高创造度英文诗
    print("\n=== 生成高创造度英文诗 ===")
    topic = "starry night"
    poetry = generate_creative_poetry(topic)
    print(f"主题: {topic}\n")
    print(poetry)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


⚠️  跳过第920行：内容为空或格式错误
⚠️  跳过第946行：内容为空或格式错误
⚠️  跳过第4092行：内容为空或格式错误
⚠️  跳过第4192行：内容为空或格式错误
⚠️  跳过第4194行：内容为空或格式错误
⚠️  跳过第4199行：内容为空或格式错误
⚠️  跳过第4200行：内容为空或格式错误
⚠️  跳过第4224行：内容为空或格式错误
⚠️  跳过第4242行：内容为空或格式错误
⚠️  跳过第4243行：内容为空或格式错误
⚠️  跳过第4252行：内容为空或格式错误
⚠️  跳过第4254行：内容为空或格式错误
⚠️  跳过第4301行：内容为空或格式错误
⚠️  跳过第4302行：内容为空或格式错误
⚠️  跳过第4303行：内容为空或格式错误
⚠️  跳过第4457行：内容为空或格式错误
⚠️  跳过第4458行：内容为空或格式错误
⚠️  跳过第4459行：内容为空或格式错误
⚠️  跳过第4541行：内容为空或格式错误
⚠️  跳过第4602行：内容为空或格式错误
⚠️  跳过第4603行：内容为空或格式错误
⚠️  跳过第4762行：内容为空或格式错误
⚠️  跳过第4763行：内容为空或格式错误
⚠️  跳过第5447行：内容为空或格式错误
⚠️  跳过第5452行：内容为空或格式错误
⚠️  跳过第5595行：内容为空或格式错误
⚠️  跳过第5597行：内容为空或格式错误
⚠️  跳过第5675行：内容为空或格式错误
⚠️  跳过第5677行：内容为空或格式错误
⚠️  跳过第5789行：内容为空或格式错误
⚠️  跳过第5790行：内容为空或格式错误
⚠️  跳过第5791行：内容为空或格式错误
⚠️  跳过第5792行：内容为空或格式错误
⚠️  跳过第5829行：内容为空或格式错误
⚠️  跳过第5836行：内容为空或格式错误
⚠️  跳过第5837行：内容为空或格式错误
⚠️  跳过第5853行：内容为空或格式错误
⚠️  跳过第5861行：内容为空或格式错误
⚠️  跳过第5969行：内容为空或格式错误
⚠️  跳过第5970行：内容为空或格式错误
⚠️  跳过第5989行：内容为空或格式错误
⚠️  跳过第5990行：内容为空或格式错误
⚠️  跳过第6264行：内容为空或格式错误
⚠️  跳过第6334行：

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.34it/s]


trainable params: 5,046,272 || all params: 7,620,662,784 || trainable%: 0.0662

⚠️  Step 0 出现NaN Loss，跳过该步

⚠️  Step 1 出现NaN Loss，跳过该步

⚠️  Step 2 出现NaN Loss，跳过该步

⚠️  Step 3 出现NaN Loss，跳过该步

⚠️  Step 4 出现NaN Loss，跳过该步

⚠️  Step 5 出现NaN Loss，跳过该步

⚠️  Step 6 出现NaN Loss，跳过该步

⚠️  Step 7 出现NaN Loss，跳过该步

⚠️  Step 8 出现NaN Loss，跳过该步

⚠️  Step 9 出现NaN Loss，跳过该步

⚠️  Step 10 出现NaN Loss，跳过该步

⚠️  Step 11 出现NaN Loss，跳过该步

⚠️  Step 12 出现NaN Loss，跳过该步

⚠️  Step 13 出现NaN Loss，跳过该步

⚠️  Step 14 出现NaN Loss，跳过该步

⚠️  Step 15 出现NaN Loss，跳过该步

⚠️  Step 16 出现NaN Loss，跳过该步

⚠️  Step 17 出现NaN Loss，跳过该步

⚠️  Step 18 出现NaN Loss，跳过该步

⚠️  Step 19 出现NaN Loss，跳过该步

⚠️  Step 20 出现NaN Loss，跳过该步

⚠️  Step 21 出现NaN Loss，跳过该步

⚠️  Step 22 出现NaN Loss，跳过该步

⚠️  Step 23 出现NaN Loss，跳过该步

⚠️  Step 24 出现NaN Loss，跳过该步

⚠️  Step 25 出现NaN Loss，跳过该步

⚠️  Step 26 出现NaN Loss，跳过该步

⚠️  Step 27 出现NaN Loss，跳过该步

⚠️  Step 28 出现NaN Loss，跳过该步

⚠️  Step 29 出现NaN Loss，跳过该步

⚠️  Step 30 出现NaN Loss，跳过该步

⚠️  Step 31 出现NaN Loss，跳过该步

⚠️

KeyboardInterrupt: 

In [2]:
# %%
import json
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import torch.nn as nn
import os
import time
import random
import nltk

# -------------------------
# 0. 基础配置（优化生成参数）
# -------------------------
# 缓存配置
os.environ["TRANSFORMERS_CACHE"] = "/root/autodl-tmp/hf_cache"
os.environ["HF_HOME"] = "/root/autodl-tmp/hf_cache"
torch.hub.set_dir("/root/autodl-tmp/torch_cache")
os.environ["TORCH_HOME"] = "/root/autodl-tmp/torch_cache"
os.makedirs("/root/autodl-tmp/offload", exist_ok=True)

# 调试专用配置（核心增加生成参数）
CONFIG = {
    "model_name_or_path": "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b",
    "data_path": "/root/autodl-tmp/Pro/poetry_sft.jsonl",
    "output_dir": "/root/autodl-tmp/Pro/qwen2.5-sft-debug",
    "num_train_epochs": 2,  # 保持原训练轮数
    "batch_size": 1,
    "gradient_accumulation_steps": 1,
    "learning_rate": 1e-5,
    "max_seq_length": 128,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "offload_folder": "/root/autodl-tmp/offload",
    # 新增生成专用配置
    "gen_max_new_tokens": 200,  # 仅新增200个token（避免包含prompt长度）
    "gen_temperature": 0.85,
    "gen_top_k": 30,
    "gen_top_p": 0.92,
    "gen_repetition_penalty": 1.15
}

# -------------------------
# 1. 加载Tokenizer（优化特殊token处理）
# -------------------------
tokenizer_file = os.path.join(CONFIG["model_name_or_path"], "tokenizer.json")
assert os.path.exists(tokenizer_file), f"Tokenizer文件不存在: {tokenizer_file}"
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
# 强制对齐Qwen2.5特殊token
tokenizer.eos_token = "<|endoftext|>"
tokenizer.bos_token = "<|startoftext|>"
tokenizer.pad_token = tokenizer.eos_token
# 添加对话专用特殊token
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<|im_start|>", "<|im_end|>"]
})
print("✅ Tokenizer加载完成")

# -------------------------
# 2. 仅加载第一条数据的Dataset（无修改）
# -------------------------
class SingleSamplePoetryDataset(Dataset):
    def __init__(self, jsonl_path, tokenizer, max_len=128):
        self.tokenizer = tokenizer
        self.max_len = max_len
        # 仅加载第一条有效数据
        self.single_sample = self._load_first_valid_sample(jsonl_path)
        assert self.single_sample is not None, "无有效数据！"
        print(f"✅ 加载第一条有效数据：")
        print(f"用户输入: {self.single_sample['user'][:50]}...")
        print(f"诗歌内容: {self.single_sample['assistant'][:50]}...")

    def _load_first_valid_sample(self, jsonl_path):
        """仅加载第一条有效数据"""
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    sample = json.loads(line)
                    # 校验数据格式
                    if len(sample.get("messages", [])) < 2:
                        continue
                    user_text = sample["messages"][0]["content"].strip()
                    assistant_text = sample["messages"][1]["content"].strip()
                    if user_text and assistant_text:
                        return {"user": user_text, "assistant": assistant_text}
                except Exception as e:
                    print(f"跳过无效行: {e}")
                    continue
        return None

    def __len__(self):
        """强制长度为1，仅返回第一条数据"""
        return 1

    def __getitem__(self, idx):
        """无论idx是多少，都返回第一条数据"""
        sample = self.single_sample
        # 构建prompt
        prompt = f"<|im_start|>user\n{sample['user']}<|im_end|>\n<|im_start|>assistant\n{sample['assistant']}<|im_end|>"
        
        # 编码（详细日志）
        encoding = self.tokenizer(
            prompt,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            padding_side="right",
            return_attention_mask=True,
            return_tensors="pt"
        )
        input_ids = encoding.input_ids.squeeze(0).long()
        attention_mask = encoding.attention_mask.squeeze(0).long()

        # 构建标签（仅计算assistant部分loss）
        user_prompt = f"<|im_start|>user\n{sample['user']}<|im_end|>\n<|im_start|>assistant\n"
        user_len = len(self.tokenizer.encode(user_prompt, truncation=True, max_length=self.max_len))
        labels = torch.full_like(input_ids, -100)  # 忽略user部分
        # 仅标注assistant前50个token（减少计算）
        labels[user_len:min(user_len+50, self.max_len)] = input_ids[user_len:min(user_len+50, self.max_len)]

        # 调试：打印关键维度
        print(f"\n📌 数据维度校验:")
        print(f"Input IDs长度: {len(input_ids)}")
        print(f"User部分长度: {user_len}")
        print(f"有效标签数: {torch.sum(labels != -100).item()}")
        print(f"标签示例: {labels[user_len:user_len+10]}")

        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask
        }

# 加载单样本数据集
dataset = SingleSamplePoetryDataset(CONFIG["data_path"], tokenizer, CONFIG["max_seq_length"])
# 强制只加载一条数据（shuffle=False避免打乱）
loader = DataLoader(
    dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=False,
    num_workers=0,
    drop_last=True
)
assert len(loader) == 1, "数据加载器长度不为1！"

# -------------------------
# 3. 加载模型（FP32+极简LoRA）（无修改）
# -------------------------
print("\n📌 加载模型...")
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["model_name_or_path"],
    torch_dtype=torch.float32,  # 强制FP32，避免数值溢出
    device_map="auto",
    offload_folder=CONFIG["offload_folder"],
    attn_implementation="eager",  # 禁用flash attention
    low_cpu_mem_usage=True
)
model.train()

# 极简LoRA配置（仅训练q/v投影）
lora_cfg = LoraConfig(
    r=2,  # 极小的r值，减少计算
    lora_alpha=8,
    lora_dropout=0.01,
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
    bias="none"
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

# -------------------------
# 4. 优化器+损失函数（无修改）
# -------------------------
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=CONFIG["learning_rate"],
    eps=1e-8,
    weight_decay=0.001
)
loss_fn = nn.CrossEntropyLoss(ignore_index=-100, reduction="mean")

# -------------------------
# 5. 单样本训练循环（调整梯度裁剪阈值）
# -------------------------
def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

print("\n📌 开始单样本训练...")
for epoch in range(CONFIG["num_train_epochs"]):
    epoch_start = time.time()
    total_loss = 0.0
    optimizer.zero_grad()

    # 仅遍历唯一的一个batch（第一条数据）
    for step, batch in enumerate(loader):
        print(f"\n===== Epoch {epoch+1} | Step {step+1} =====")
        
        # 加载数据到设备
        input_ids = batch["input_ids"].to(CONFIG["device"])
        labels = batch["labels"].to(CONFIG["device"])
        attention_mask = batch["attention_mask"].to(CONFIG["device"])

        # 调试：打印数据设备和数值范围
        print(f"数据设备: {input_ids.device}")
        print(f"Input IDs范围: {input_ids.min().item()} ~ {input_ids.max().item()}")
        print(f"有效标签范围: {labels[labels != -100].min().item()} ~ {labels[labels != -100].max().item()}")

        # 前向传播（带梯度检测）
        with torch.autograd.detect_anomaly():  # 检测梯度NaN/Inf
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            logits = outputs.logits
            loss = outputs.loss

        # 调试：打印关键数值
        print(f"Logits形状: {logits.shape}")
        print(f"Logits均值: {logits.mean().item():.4f} | 最大值: {logits.max().item():.4f} | 最小值: {logits.min().item():.4f}")
        print(f"原始Loss值: {loss.item():.4f}")

        # 检查Loss是否为NaN
        if torch.isnan(loss):
            print("❌ Loss为NaN！")
            # 打印更详细的logits信息
            print(f"Logits是否含NaN: {torch.isnan(logits).any().item()}")
            print(f"Logits是否含Inf: {torch.isinf(logits).any().item()}")
            break

        # 反向传播（带梯度检测）
        loss.backward()

        # 调试：打印梯度信息（调整裁剪阈值到1.5）
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.5)
        print(f"梯度范数: {grad_norm.item():.4f}")

        # 更新参数
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    # 打印Epoch总结
    epoch_time = format_time(time.time() - epoch_start)
    avg_loss = total_loss / len(loader)
    print(f"\n✅ Epoch {epoch+1} 完成 | 平均Loss: {avg_loss:.4f} | 耗时: {epoch_time}")

# -------------------------
# 6. 保存模型（调试用）（无修改）
# -------------------------
model.save_pretrained(CONFIG["output_dir"])
tokenizer.save_pretrained(CONFIG["output_dir"])
print(f"\n🎉 单样本训练完成！模型保存至: {CONFIG['output_dir']}")

# -------------------------
# 7. 验证生成（核心优化：解决不完整+残留标签）
# -------------------------
def generate_single_sample():
    model.eval()
    sample = dataset.single_sample
    # 构建纯prompt（不含答案）
    prompt = f"<|im_start|>user\n{sample['user']}<|im_end|>\n<|im_start|>assistant\n"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(CONFIG["device"])
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=CONFIG["gen_max_new_tokens"],  # 仅新增token数（关键！）
            do_sample=True,
            temperature=CONFIG["gen_temperature"],
            top_k=CONFIG["gen_top_k"],
            top_p=CONFIG["gen_top_p"],
            repetition_penalty=CONFIG["gen_repetition_penalty"],
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,  # 遇到eos自动停止
            num_return_sequences=1
        )
    
    # 优化解码逻辑：移除特殊token+仅保留assistant部分
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    # 截取assistant之后的内容
    if "<|im_start|>assistant\n" in generated_text:
        generated_poem = generated_text.split("<|im_start|>assistant\n")[1]
        # 移除末尾的eos token
        generated_poem = generated_poem.replace("<|endoftext|>", "").replace("<|im_end|>", "").strip()
    else:
        generated_poem = generated_text.strip()
    
    print("\n📌 生成结果验证:")
    print(f"主题: {sample['user']}")
    print("生成诗歌:")
    print(generated_poem)

generate_single_sample()

✅ Tokenizer加载完成
✅ 加载第一条有效数据：
用户输入: Write a poem in the style of Michelle Menting....
诗歌内容: Dog bone, stapler,
cribbage board, garlic press
  ...

📌 加载模型...


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.48s/it]
/tmp/ipykernel_5568/1839768931.py:217: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():  # 检测梯度NaN/Inf


trainable params: 630,784 || all params: 7,616,247,296 || trainable%: 0.0083

📌 开始单样本训练...

📌 数据维度校验:
Input IDs长度: 128
User部分长度: 19
有效标签数: 50
标签示例: tensor([48940, 17240,    11, 35014,  1536,   345,   740,  6066,   424,  4479])

===== Epoch 1 | Step 1 =====
数据设备: cuda:0
Input IDs范围: 11 ~ 151645
有效标签范围: 11 ~ 78901
Logits形状: torch.Size([1, 128, 152064])
Logits均值: -2.4702 | 最大值: 26.6663 | 最小值: -28.6940
原始Loss值: 4.8307
梯度范数: 1.2347

✅ Epoch 1 完成 | 平均Loss: 4.8307 | 耗时: 00:00:00

📌 数据维度校验:
Input IDs长度: 128
User部分长度: 19
有效标签数: 50
标签示例: tensor([48940, 17240,    11, 35014,  1536,   345,   740,  6066,   424,  4479])

===== Epoch 2 | Step 1 =====
数据设备: cuda:0
Input IDs范围: 11 ~ 151645
有效标签范围: 11 ~ 78901


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Logits形状: torch.Size([1, 128, 152064])
Logits均值: -2.4705 | 最大值: 26.6685 | 最小值: -28.6951
原始Loss值: 4.8281
梯度范数: 1.2484

✅ Epoch 2 完成 | 平均Loss: 4.8281 | 耗时: 00:00:00

🎉 单样本训练完成！模型保存至: /root/autodl-tmp/Pro/qwen2.5-sft-debug

📌 生成结果验证:
主题: Write a poem in the style of Michelle Menting.
生成诗歌:
In the stillness of dawn's early light,
I find solace, as I always do.
The world awakes to gentle sight,
Where whispers weave through morning dew.

A sparrow chirps its lullaby,
As if it knows what lies ahead.
Its song is soft and tenderly,
A melody for weary heads.

Across the fields where golden hues lie,
And wildflowers nod under blue skies.
My heart beats with an ancient sigh—
A rhythm known from times agone.

For every step upon this earth,
Is tread by souls who sought before;
To chase their dreams, though hearts might hurt,
And hold on tight against life's shore.

So here we walk, hand in hand again,
Through stories old yet fresh and new.
Each moment holds both joy and pain,
But together, none can

In [1]:
# %%
import json
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import torch.nn as nn
import os
import time
import random
import nltk

# -------------------------
# 0. 全量训练配置（平衡效率与稳定性）
# -------------------------
# 缓存配置
os.environ["TRANSFORMERS_CACHE"] = "/root/autodl-tmp/hf_cache"
os.environ["HF_HOME"] = "/root/autodl-tmp/hf_cache"
torch.hub.set_dir("/root/autodl-tmp/torch_cache")
os.environ["TORCH_HOME"] = "/root/autodl-tmp/torch_cache"
os.makedirs("/root/autodl-tmp/offload", exist_ok=True)

# 全量训练配置
CONFIG = {
    "model_name_or_path": "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b",
    "data_path": "/root/autodl-tmp/Pro/poetry_sft.jsonl",
    "output_dir": "/root/autodl-tmp/Pro/qwen2.5-sft-full",
    "num_train_epochs": 3,  # 全量数据训练3轮
    "batch_size": 2,  # 按需调整（根据GPU显存）
    "gradient_accumulation_steps": 2,  # 梯度累积，等效增大批次
    "learning_rate": 1e-5,
    "max_seq_length": 256,  # 适配更长的诗歌文本
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "offload_folder": "/root/autodl-tmp/offload",
    # 生成专用配置
    "gen_max_new_tokens": 200,
    "gen_temperature": 0.85,
    "gen_top_k": 30,
    "gen_top_p": 0.92,
    "gen_repetition_penalty": 1.15,
    # 训练监控
    "print_every_steps": 10,  # 每10步打印一次Loss
}

# -------------------------
# 1. 加载Tokenizer（兼容全量数据）
# -------------------------
tokenizer_file = os.path.join(CONFIG["model_name_or_path"], "tokenizer.json")
assert os.path.exists(tokenizer_file), f"Tokenizer文件不存在: {tokenizer_file}"
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
# Qwen2.5专用特殊token配置
tokenizer.eos_token = "<|endoftext|>"
tokenizer.bos_token = "<|startoftext|>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<|im_start|>", "<|im_end|>"]
})
print("✅ Tokenizer加载完成")

# -------------------------
# 2. 全量数据Dataset（加载所有有效诗歌）
# -------------------------
class PoetryDataset(Dataset):
    def __init__(self, jsonl_path, tokenizer, max_len=256):
        self.tokenizer = tokenizer
        self.max_len = max_len
        # 加载所有有效数据
        self.data = self._load_all_valid_samples(jsonl_path)
        assert len(self.data) > 0, f"数据路径 {jsonl_path} 下无有效数据！"
        print(f"✅ 加载有效诗歌数据: {len(self.data)} 条")

    def _load_all_valid_samples(self, jsonl_path):
        """加载所有有效诗歌数据"""
        valid_data = []
        if not os.path.exists(jsonl_path):
            raise FileNotFoundError(f"数据文件不存在: {jsonl_path}")
        
        with open(jsonl_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            for line_idx, line in enumerate(lines):
                line = line.strip()
                if not line:
                    continue
                try:
                    sample = json.loads(line)
                    # 校验messages结构和内容
                    if len(sample.get("messages", [])) < 2:
                        print(f"⚠️  跳过第{line_idx+1}行：messages格式错误")
                        continue
                    user_text = sample["messages"][0]["content"].strip()
                    assistant_text = sample["messages"][1]["content"].strip()
                    if not user_text or not assistant_text:
                        print(f"⚠️  跳过第{line_idx+1}行：内容为空")
                        continue
                    valid_data.append({
                        "user": user_text,
                        "assistant": assistant_text
                    })
                except json.JSONDecodeError:
                    print(f"⚠️  跳过第{line_idx+1}行：JSON解析失败")
                except Exception as e:
                    print(f"⚠️  跳过第{line_idx+1}行：{str(e)}")
        return valid_data

    def __len__(self):
        """返回全量数据长度"""
        return len(self.data)

    def __getitem__(self, idx):
        """按索引加载单条数据"""
        sample = self.data[idx]
        # 构建Qwen2.5标准对话prompt
        prompt = f"<|im_start|>user\n{sample['user']}<|im_end|>\n<|im_start|>assistant\n{sample['assistant']}<|im_end|>"
        
        # 编码（适配长序列）
        encoding = self.tokenizer(
            prompt,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            padding_side="right",
            return_attention_mask=True,
            return_tensors="pt"
        )
        input_ids = encoding.input_ids.squeeze(0).long()
        attention_mask = encoding.attention_mask.squeeze(0).long()

        # 构建标签（仅计算assistant部分Loss）
        user_prompt = f"<|im_start|>user\n{sample['user']}<|im_end|>\n<|im_start|>assistant\n"
        user_len = len(self.tokenizer.encode(user_prompt, truncation=True, max_length=self.max_len))
        labels = torch.full_like(input_ids, -100)  # user部分忽略
        # 标注assistant全部有效token（适配长序列）
        labels[user_len:min(user_len + self.max_len - user_len, self.max_len)] = input_ids[user_len:min(user_len + self.max_len - user_len, self.max_len)]

        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask,
            "user_text": sample["user"],  # 保留用户输入，用于生成验证
            "assistant_text": sample["assistant"]  # 保留参考答案
        }

# 加载全量数据集
dataset = PoetryDataset(CONFIG["data_path"], tokenizer, CONFIG["max_seq_length"])
# 全量数据DataLoader（打乱+批次加载）
train_loader = DataLoader(
    dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=True,  # 打乱数据，提升泛化性
    num_workers=0,  # 禁用多进程，避免数据加载异常
    drop_last=True  # 丢弃最后不完整批次
)
total_steps = len(train_loader) * CONFIG["num_train_epochs"]
print(f"📌 全量训练配置：")
print(f"- 总数据量：{len(dataset)} 条")
print(f"- 每Epoch步数：{len(train_loader)}")
print(f"- 总训练步数：{total_steps}")
print(f"- 有效批次大小：{CONFIG['batch_size'] * CONFIG['gradient_accumulation_steps']}")

# -------------------------
# 3. 加载模型（FP32+LoRA，全量训练稳定）
# -------------------------
print("\n📌 加载Qwen2.5-7B模型...")
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["model_name_or_path"],
    torch_dtype=torch.float32,  # 保持FP32稳定，全量训练不溢出
    device_map="auto",
    offload_folder=CONFIG["offload_folder"],
    attn_implementation="eager",  # 禁用flash attention，提升稳定性
    low_cpu_mem_usage=True
)
model.train()

# LoRA配置（适度增大r值，提升全量数据拟合能力）
lora_cfg = LoraConfig(
    r=4,  # 比单样本调试增大，适配全量数据
    lora_alpha=16,
    lora_dropout=0.01,
    target_modules=["q_proj", "v_proj"],  # 仅训练关键投影层
    task_type="CAUSAL_LM",
    bias="none"
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

# -------------------------
# 4. 优化器配置（适配全量训练）
# -------------------------
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=CONFIG["learning_rate"],
    eps=1e-8,
    weight_decay=0.001,
    betas=(0.9, 0.999)
)
# 损失函数（忽略padding部分）
loss_fn = nn.CrossEntropyLoss(ignore_index=-100, reduction="mean")

# -------------------------
# 5. 训练辅助函数
# -------------------------
def format_time(seconds):
    """格式化时间输出"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

def generate_validation_sample(model, tokenizer, dataset, config):
    """随机选1条数据验证生成效果"""
    model.eval()
    # 随机选1条数据
    sample_idx = random.randint(0, len(dataset)-1)
    sample = dataset.data[sample_idx]
    prompt = f"<|im_start|>user\n{sample['user']}<|im_end|>\n<|im_start|>assistant\n"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(config["device"])
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=config["gen_max_new_tokens"],
            do_sample=True,
            temperature=config["gen_temperature"],
            top_k=config["gen_top_k"],
            top_p=config["gen_top_p"],
            repetition_penalty=config["gen_repetition_penalty"],
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,
            num_return_sequences=1
        )
    
    # 解码并清理生成结果
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    if "<|im_start|>assistant\n" in generated_text:
        generated_poem = generated_text.split("<|im_start|>assistant\n")[1]
        generated_poem = generated_poem.replace("<|endoftext|>", "").replace("<|im_end|>", "").strip()
    else:
        generated_poem = generated_text.strip()
    
    # 打印验证结果
    print("\n📝 生成验证（随机样本）:")
    print(f"主题: {sample['user'][:50]}...")
    print("生成诗歌:")
    print(generated_poem)
    print("-" * 50)
    model.train()  # 回到训练模式

# -------------------------
# 6. 全量训练主循环
# -------------------------
print("\n🚀 开始全量数据训练...")
global_step = 0
total_train_time = 0.0
avg_loss_list = []

for epoch in range(CONFIG["num_train_epochs"]):
    epoch_start = time.time()
    epoch_loss = 0.0
    optimizer.zero_grad()

    for step, batch in enumerate(train_loader):
        step_start = time.time()
        global_step += 1

        # 加载批次数据到设备
        input_ids = batch["input_ids"].to(CONFIG["device"], non_blocking=True)
        labels = batch["labels"].to(CONFIG["device"], non_blocking=True)
        attention_mask = batch["attention_mask"].to(CONFIG["device"], non_blocking=True)

        # 前向传播（梯度检测）
        with torch.autograd.detect_anomaly():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            # 梯度累积：Loss除以累积步数
            loss = loss / CONFIG["gradient_accumulation_steps"]

        # 反向传播
        loss.backward()

        # 梯度裁剪（防止爆炸）
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.5)

        # 累积步数达标后更新参数
        if (step + 1) % CONFIG["gradient_accumulation_steps"] == 0:
            optimizer.step()
            optimizer.zero_grad()

        # 累计Loss
        epoch_loss += loss.item() * CONFIG["gradient_accumulation_steps"]
        avg_loss_list.append(loss.item() * CONFIG["gradient_accumulation_steps"])

        # 打印训练进度
        if global_step % CONFIG["print_every_steps"] == 0:
            step_time = time.time() - step_start
            avg_loss = sum(avg_loss_list[-CONFIG["print_every_steps"]:]) / CONFIG["print_every_steps"]
            print(f"[Epoch {epoch+1}/{CONFIG['num_train_epochs']}] "
                  f"Step {global_step}/{total_steps} | "
                  f"Avg Loss: {avg_loss:.4f} | "
                  f"Grad Norm: {grad_norm.item():.4f} | "
                  f"Step Time: {step_time:.2f}s")

    # Epoch结束处理
    epoch_time = format_time(time.time() - epoch_start)
    total_train_time += time.time() - epoch_start
    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"\n✅ Epoch {epoch+1} 完成 | "
          f"Avg Loss: {avg_epoch_loss:.4f} | "
          f"Epoch Time: {epoch_time} | "
          f"Total Time: {format_time(total_train_time)}")

    # 每个Epoch结束后验证生成效果
    generate_validation_sample(model, tokenizer, dataset, CONFIG)

# -------------------------
# 7. 保存最终模型
# -------------------------
print("\n📦 保存训练完成模型...")
model.save_pretrained(CONFIG["output_dir"])
tokenizer.save_pretrained(CONFIG["output_dir"])
print(f"✅ 模型已保存至: {CONFIG['output_dir']}")

# -------------------------
# 8. 最终生成验证（随机选3条数据）
# -------------------------
print("\n🎯 最终生成验证（随机3条数据）:")
for i in range(3):
    generate_validation_sample(model, tokenizer, dataset, CONFIG)

print("\n🎉 全量训练完成！")

/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


✅ Tokenizer加载完成
⚠️  跳过第920行：内容为空
⚠️  跳过第946行：内容为空
⚠️  跳过第4092行：内容为空
⚠️  跳过第4192行：内容为空
⚠️  跳过第4194行：内容为空
⚠️  跳过第4199行：内容为空
⚠️  跳过第4200行：内容为空
⚠️  跳过第4224行：内容为空
⚠️  跳过第4242行：内容为空
⚠️  跳过第4243行：内容为空
⚠️  跳过第4252行：内容为空
⚠️  跳过第4254行：内容为空
⚠️  跳过第4301行：内容为空
⚠️  跳过第4302行：内容为空
⚠️  跳过第4303行：内容为空
⚠️  跳过第4457行：内容为空
⚠️  跳过第4458行：内容为空
⚠️  跳过第4459行：内容为空
⚠️  跳过第4541行：内容为空
⚠️  跳过第4602行：内容为空
⚠️  跳过第4603行：内容为空
⚠️  跳过第4762行：内容为空
⚠️  跳过第4763行：内容为空
⚠️  跳过第5447行：内容为空
⚠️  跳过第5452行：内容为空
⚠️  跳过第5595行：内容为空
⚠️  跳过第5597行：内容为空
⚠️  跳过第5675行：内容为空
⚠️  跳过第5677行：内容为空
⚠️  跳过第5789行：内容为空
⚠️  跳过第5790行：内容为空
⚠️  跳过第5791行：内容为空
⚠️  跳过第5792行：内容为空
⚠️  跳过第5829行：内容为空
⚠️  跳过第5836行：内容为空
⚠️  跳过第5837行：内容为空
⚠️  跳过第5853行：内容为空
⚠️  跳过第5861行：内容为空
⚠️  跳过第5969行：内容为空
⚠️  跳过第5970行：内容为空
⚠️  跳过第5989行：内容为空
⚠️  跳过第5990行：内容为空
⚠️  跳过第6264行：内容为空
⚠️  跳过第6334行：内容为空
⚠️  跳过第6335行：内容为空
⚠️  跳过第6367行：内容为空
⚠️  跳过第6368行：内容为空
⚠️  跳过第6369行：内容为空
⚠️  跳过第6370行：内容为空
⚠️  跳过第6371行：内容为空
⚠️  跳过第6372行：内容为空
⚠️  跳过第6373行：内容为空
⚠️  跳过第6374行：内容为空
⚠️  跳过第6375行：内容为空
⚠️  跳过第6376行：内

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.41s/it]


trainable params: 1,261,568 || all params: 7,616,878,080 || trainable%: 0.0166

🚀 开始全量数据训练...


/tmp/ipykernel_6091/124602864.py:273: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():


[Epoch 1/3] Step 10/20628 | Avg Loss: 4.4161 | Grad Norm: 1.5411 | Step Time: 0.83s
[Epoch 1/3] Step 20/20628 | Avg Loss: 6.2027 | Grad Norm: 5.2924 | Step Time: 0.65s
[Epoch 1/3] Step 30/20628 | Avg Loss: 6.3657 | Grad Norm: 5.8603 | Step Time: 0.65s
[Epoch 1/3] Step 40/20628 | Avg Loss: 6.9830 | Grad Norm: 9.0553 | Step Time: 0.68s
[Epoch 1/3] Step 50/20628 | Avg Loss: 5.6384 | Grad Norm: 5.7839 | Step Time: 0.65s
[Epoch 1/3] Step 60/20628 | Avg Loss: 4.5480 | Grad Norm: 5.5153 | Step Time: 0.65s
[Epoch 1/3] Step 70/20628 | Avg Loss: 6.2103 | Grad Norm: 1.5747 | Step Time: 0.67s
[Epoch 1/3] Step 80/20628 | Avg Loss: 5.6445 | Grad Norm: 8.2896 | Step Time: 0.65s
[Epoch 1/3] Step 90/20628 | Avg Loss: 5.3434 | Grad Norm: 5.1194 | Step Time: 0.66s
[Epoch 1/3] Step 100/20628 | Avg Loss: 6.2313 | Grad Norm: 15.5805 | Step Time: 0.65s
[Epoch 1/3] Step 110/20628 | Avg Loss: 5.6932 | Grad Norm: 8.3427 | Step Time: 0.66s
[Epoch 1/3] Step 120/20628 | Avg Loss: 5.2875 | Grad Norm: 3.6313 | Step 

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



✅ Epoch 1 完成 | Avg Loss: 2.9608 | Epoch Time: 01:16:50 | Total Time: 01:16:50

📝 生成验证（随机样本）:
主题: Write a poem using the theme: Living,Death,Sorrow ...
生成诗歌:
In this house on its hilltop,
The windows open to the sea;
The curtains have been hung.
On the sill is an odd old lamp
With a long stem and black brass handle—
It was from some stranger’s room.
Outside in the cold darkness,
A woman weeps.
She has come from another world.
No one knows where she comes from.
She may be from the country
Beyond the river that flows through the hills
Into the sea.
But she says the rain falls there
Through the trees all day
And her heart is like a flower
That opens only at night.
She cries for no one.
What are your tears about? 
I will not answer you.
You must go back home now
To wherever it is you live.
There is someone who waits for you:
Her face is pale with sorrow.
Who is this? I do not know.
--------------------------------------------------
[Epoch 2/3] Step 6880/20628 | Avg Loss: 2.9640 | Grad Norm

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# -------------------------
# 1. 加载你微调好的模型
# -------------------------
MODEL_PATH = "/root/autodl-tmp/Pro/qwen2.5-sft-full"   # ← 改成你的路径

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
model.eval()

# -------------------------
# 2. 生成函数
# -------------------------
def generate_by_prompt(user_prompt: str, max_new_tokens=200):
    """
    user_prompt: 你自己随便输入的 prompt
    （可以包含 <|style:xxx|>, 也可以纯英文描述）
    """
    prompt = (
        f"<|im_start|>user\n"
        f"{user_prompt}\n"
        f"<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.9,
            top_p=0.92,
            top_k=40,
            repetition_penalty=1.15,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    if "<|im_start|>assistant" in text:
        poem = text.split("<|im_start|>assistant")[-1]
    else:
        poem = text

    poem = poem.replace("<|endoftext|>", "").replace("<|im_end|>", "").strip()
    return poem


# -------------------------
# 3. 用户输入
# -------------------------
print("🎤 请输入你的生成 prompt（可使用风格标签或英文关键词）:")
print("📌 示例： <|style:byron|> Write a poem about moonlight and sorrow.")
print("输入后按回车：\n")

user_input = input("Your prompt: ")

# -------------------------
# 4. 生成
# -------------------------
print("\n✨ Generated Poem:\n")
result = generate_by_prompt(user_input)
print(result)


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.57it/s]


🎤 请输入你的生成 prompt（可使用风格标签或英文关键词）:
📌 示例： <|style:byron|> Write a poem about moonlight and sorrow.
输入后按回车：



Your prompt:  write a poem in the style of virginia wolf, with keyword: willow, wind, fluffy, charmed.



✨ Generated Poem:

The fluffiness of willows and the fluttering of their leaves, The gauze curtains that wave without any movement from within, The droning of bees, The whirring of motor-buses, The roar of traffic—these are all things I want to do away with. If I could write my book as simply and as naturally as one breathes, I would forget about it for years, then suddenly feel some strange emotion rising up within me which would make everything else disappear, would leave me alone on an eminence where I should see only a few tall trees swaying gently in the wind, perhaps two or three white clouds floating far out over the sea. Then if I were still possessed by this feeling, I might begin to think again of my story. As it is, I can never remember anything when there’s a lot going on; but, like most people, I have to get used to living among other people before they start getting in the way. And


In [1]:
# %%
import json
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import (
    PreTrainedTokenizerFast, AutoModelForCausalLM,
    AutoTokenizer, AutoModelForSequenceClassification
)
from peft import LoraConfig, get_peft_model
import torch.nn as nn
import os
import time
import random
import warnings
warnings.filterwarnings("ignore")

# -------------------------
# 0. 全量训练配置（集成ADV Loss）
# -------------------------
# 缓存配置
os.environ["TRANSFORMERS_CACHE"] = "/root/autodl-tmp/hf_cache"
os.environ["HF_HOME"] = "/root/autodl-tmp/hf_cache"
torch.hub.set_dir("/root/autodl-tmp/torch_cache")
os.environ["TORCH_HOME"] = "/root/autodl-tmp/torch_cache"
os.makedirs("/root/autodl-tmp/offload", exist_ok=True)

# 关键：启用CUDA同步执行，解决设备端断言错误
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

# 全量训练配置（集成对抗训练）
CONFIG = {
    "model_name_or_path": "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b",
    "data_path": "/root/autodl-tmp/Pro/poetry_sft.jsonl",
    "output_dir": "/root/autodl-tmp/Pro/qwen2.5-sft-full-adv",
    "num_train_epochs": 2,  # 全量数据训练3轮
    "batch_size": 2,  # 按需调整（根据GPU显存）
    "gradient_accumulation_steps": 2,  # 梯度累积，等效增大批次
    "learning_rate": 1e-5,
    "max_seq_length": 256,  # 适配更长的诗歌文本
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "offload_folder": "/root/autodl-tmp/offload",
    # 生成专用配置
    "gen_max_new_tokens": 100,
    "gen_temperature": 0.85,
    "gen_top_k": 30,
    "gen_top_p": 0.92,
    "gen_repetition_penalty": 1.15,
    # 对抗训练配置
    "detector_model_path": "/root/autodl-tmp/Pro/models--roberta-base-openai-detector/snapshots/6cba99c003b711c7fe94f8a3aa2be35a792cb6fa",
    "detector_every_steps": 100,  # 每8步计算一次对抗损失
    "lambda_adv_max": 0.25,     # 对抗损失最大权重
    "lambda_adv_base": 0.02,    # 对抗损失基础权重
    "lambda_adv_step": 0.015,   # 每epoch增加的权重
    "detector_dtype": torch.float16 if torch.cuda.is_available() else torch.float32,
    # 训练监控
    "print_every_steps": 50,  # 每10步打印一次Loss
}

# -------------------------
# 1. 加载Tokenizer（兼容全量数据）
# -------------------------
tokenizer_file = os.path.join(CONFIG["model_name_or_path"], "tokenizer.json")
assert os.path.exists(tokenizer_file), f"Tokenizer文件不存在: {tokenizer_file}"
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
# Qwen2.5专用特殊token配置
tokenizer.eos_token = "<|endoftext|>"
tokenizer.bos_token = "<|startoftext|>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<|im_start|>", "<|im_end|>"]
})
print("✅ 生成模型Tokenizer加载完成")

# -------------------------
# 2. 加载AI检测器模型（用于计算ADV Loss）
# -------------------------
print("\n📌 加载AI检测器模型...")
# 加载检测器tokenizer
detector_tokenizer = AutoTokenizer.from_pretrained(
    CONFIG["detector_model_path"],
    local_files_only=True,
    padding_side="right",
    trust_remote_code=True
)

# 加载检测器模型
detector_model = AutoModelForSequenceClassification.from_pretrained(
    CONFIG["detector_model_path"],
    torch_dtype=CONFIG["detector_dtype"],
    device_map="auto",
    trust_remote_code=True
)

# 检测器设置为评估模式
detector_model.eval()
for param in detector_model.parameters():
    param.requires_grad = False  # 冻结检测器参数
print("✅ AI检测器加载完成并冻结")

@torch.no_grad()
def detector_ai_prob(text):
    """计算文本的AI生成概率（越高表示越像AI生成）"""
    # 空文本容错
    if not text or text.strip() == "":
        return 0.0
    
    # Tokenize生成文本
    det_inputs = detector_tokenizer(
        text, 
        return_tensors="pt",
        truncation=True, 
        max_length=512,
        padding="max_length"
    ).to(CONFIG["device"])
    
    # 分别处理tensor类型（关键：input_ids保持Long类型）
    for key in det_inputs.keys():
        if key == "input_ids":
            det_inputs[key] = det_inputs[key].long()
        elif key == "attention_mask":
            det_inputs[key] = det_inputs[key].to(dtype=CONFIG["detector_dtype"])
        elif det_inputs[key].dtype in [torch.int32, torch.int64]:
            det_inputs[key] = det_inputs[key].long()
        else:
            det_inputs[key] = det_inputs[key].to(dtype=CONFIG["detector_dtype"])
    
    # 检测器前向计算
    det_outputs = detector_model(**det_inputs)
    # roberta-base-openai-detector输出：[0]=人类概率，[1]=AI概率
    probabilities = torch.softmax(det_outputs.logits, dim=-1)
    ai_prob = probabilities[0, 1].item()  # AI生成的概率（0~1）
    
    return ai_prob

# -------------------------
# 3. 全量数据Dataset（加载所有有效诗歌）
# -------------------------
class PoetryDataset(Dataset):
    def __init__(self, jsonl_path, tokenizer, max_len=256):
        self.tokenizer = tokenizer
        self.max_len = max_len
        # 加载所有有效数据
        self.data = self._load_all_valid_samples(jsonl_path)
        assert len(self.data) > 0, f"数据路径 {jsonl_path} 下无有效数据！"
        print(f"✅ 加载有效诗歌数据: {len(self.data)} 条")

    def _load_all_valid_samples(self, jsonl_path):
        """加载所有有效诗歌数据"""
        valid_data = []
        if not os.path.exists(jsonl_path):
            raise FileNotFoundError(f"数据文件不存在: {jsonl_path}")
        
        with open(jsonl_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            for line_idx, line in enumerate(lines):
                line = line.strip()
                if not line:
                    continue
                try:
                    sample = json.loads(line)
                    # 校验messages结构和内容
                    if len(sample.get("messages", [])) < 2:
                        print(f"⚠️  跳过第{line_idx+1}行：messages格式错误")
                        continue
                    user_text = sample["messages"][0]["content"].strip()
                    assistant_text = sample["messages"][1]["content"].strip()
                    if not user_text or not assistant_text:
                        print(f"⚠️  跳过第{line_idx+1}行：内容为空")
                        continue
                    valid_data.append({
                        "user": user_text,
                        "assistant": assistant_text
                    })
                except json.JSONDecodeError:
                    print(f"⚠️  跳过第{line_idx+1}行：JSON解析失败")
                except Exception as e:
                    print(f"⚠️  跳过第{line_idx+1}行：{str(e)}")
        return valid_data

    def __len__(self):
        """返回全量数据长度"""
        return len(self.data)

    def __getitem__(self, idx):
        """按索引加载单条数据"""
        sample = self.data[idx]
        # 构建Qwen2.5标准对话prompt
        prompt = f"<|im_start|>user\n{sample['user']}<|im_end|>\n<|im_start|>assistant\n{sample['assistant']}<|im_end|>"
        
        # 编码（适配长序列）
        encoding = self.tokenizer(
            prompt,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            padding_side="right",
            return_attention_mask=True,
            return_tensors="pt"
        )
        input_ids = encoding.input_ids.squeeze(0).long()
        attention_mask = encoding.attention_mask.squeeze(0).long()

        # 构建标签（仅计算assistant部分Loss）
        user_prompt = f"<|im_start|>user\n{sample['user']}<|im_end|>\n<|im_start|>assistant\n"
        user_len = len(self.tokenizer.encode(user_prompt, truncation=True, max_length=self.max_len))
        labels = torch.full_like(input_ids, -100)  # user部分忽略
        # 标注assistant全部有效token（适配长序列）
        labels[user_len:min(user_len + self.max_len - user_len, self.max_len)] = input_ids[user_len:min(user_len + self.max_len - user_len, self.max_len)]

        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask,
            "user_text": sample["user"],  # 保留用户输入，用于生成验证
            "assistant_text": sample["assistant"]  # 保留参考答案
        }

# 加载全量数据集
dataset = PoetryDataset(CONFIG["data_path"], tokenizer, CONFIG["max_seq_length"])
# 全量数据DataLoader（打乱+批次加载）
train_loader = DataLoader(
    dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=True,  # 打乱数据，提升泛化性
    num_workers=0,  # 禁用多进程，避免数据加载异常
    drop_last=True  # 丢弃最后不完整批次
)
total_steps = len(train_loader) * CONFIG["num_train_epochs"]
print(f"📌 全量训练配置：")
print(f"- 总数据量：{len(dataset)} 条")
print(f"- 每Epoch步数：{len(train_loader)}")
print(f"- 总训练步数：{total_steps}")
print(f"- 有效批次大小：{CONFIG['batch_size'] * CONFIG['gradient_accumulation_steps']}")

# -------------------------
# 4. 加载生成模型（FP32+LoRA，全量训练稳定）
# -------------------------
print("\n📌 加载Qwen2.5-7B模型...")
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["model_name_or_path"],
    torch_dtype=torch.float32,  # 保持FP32稳定，全量训练不溢出
    device_map="auto",
    offload_folder=CONFIG["offload_folder"],
    attn_implementation="eager",  # 禁用flash attention，提升稳定性
    low_cpu_mem_usage=True
)
model.train()

# LoRA配置（适度增大r值，提升全量数据拟合能力）
lora_cfg = LoraConfig(
    r=4,  # 比单样本调试增大，适配全量数据
    lora_alpha=16,
    lora_dropout=0.01,
    target_modules=["q_proj", "v_proj"],  # 仅训练关键投影层
    task_type="CAUSAL_LM",
    bias="none"
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

# -------------------------
# 5. 优化器配置（适配全量训练+对抗训练）
# -------------------------
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=CONFIG["learning_rate"],
    eps=1e-8,
    weight_decay=0.001,
    betas=(0.9, 0.999)
)

# -------------------------
# 6. 训练辅助函数（集成ADV Loss）
# -------------------------
def format_time(seconds):
    """格式化时间输出"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

def generate_for_detector(model, tokenizer, user_text):
    """生成文本用于检测器评估（计算ADV Loss）"""
    # 构建标准prompt
    prompt = f"<|im_start|>user\n{user_text}<|im_end|>\n<|im_start|>assistant\n"
    
    # Tokenize
    generate_inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=CONFIG["max_seq_length"] - CONFIG["gen_max_new_tokens"]
    ).to(CONFIG["device"])
    
    input_ids = generate_inputs["input_ids"].long()
    attention_mask = generate_inputs["attention_mask"].long()
    
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=CONFIG["gen_max_new_tokens"],
            do_sample=True,
            temperature=CONFIG["gen_temperature"],
            top_k=CONFIG["gen_top_k"],
            top_p=CONFIG["gen_top_p"],
            repetition_penalty=CONFIG["gen_repetition_penalty"],
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,
            num_return_sequences=1,
            use_cache=False,
            renormalize_logits=True,
            remove_invalid_values=True
        )
    model.train()
    
    # 解码并提取生成内容
    gen_text = tokenizer.decode(
        outputs[0][input_ids.shape[-1]:],
        skip_special_tokens=True
    ).strip()
    
    # 空文本容错
    if not gen_text:
        gen_text = "无有效生成内容"
    
    return gen_text

def generate_validation_sample(model, tokenizer, dataset, config):
    """随机选1条数据验证生成效果（包含ADV Loss评估）"""
    model.eval()
    # 随机选1条数据
    sample_idx = random.randint(0, len(dataset)-1)
    sample = dataset.data[sample_idx]
    prompt = f"<|im_start|>user\n{sample['user']}<|im_end|>\n<|im_start|>assistant\n"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(config["device"])
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=config["gen_max_new_tokens"],
            do_sample=True,
            temperature=config["gen_temperature"],
            top_k=config["gen_top_k"],
            top_p=config["gen_top_p"],
            repetition_penalty=config["gen_repetition_penalty"],
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,
            num_return_sequences=1
        )
    
    # 解码并清理生成结果
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    if "<|im_start|>assistant\n" in generated_text:
        generated_poem = generated_text.split("<|im_start|>assistant\n")[1]
        generated_poem = generated_poem.replace("<|endoftext|>", "").replace("<|im_end|>", "").strip()
    else:
        generated_poem = generated_text.strip()
    
    # 计算该生成文本的AI概率（ADV Loss）
    ai_prob = detector_ai_prob(generated_poem)
    
    # 打印验证结果
    print("\n📝 生成验证（随机样本）:")
    print(f"主题: {sample['user'][:50]}...")
    print("生成诗歌:")
    print(generated_poem)
    print(f"AI判定概率 (ADV Loss): {ai_prob:.4f} (越低越像人类创作)")
    print("-" * 80)
    model.train()  # 回到训练模式

# -------------------------
# 7. 全量训练主循环（集成ADV Loss）
# -------------------------
print("\n🚀 开始全量数据对抗训练...")
global_step = 0
total_train_time = 0.0
avg_sft_loss_list = []
avg_adv_loss_list = []

for epoch in range(CONFIG["num_train_epochs"]):
    epoch_start = time.time()
    epoch_sft_loss = 0.0
    epoch_adv_loss = 0.0
    optimizer.zero_grad()

    for step, batch in enumerate(train_loader):
        step_start = time.time()
        global_step += 1

        # 加载批次数据到设备
        input_ids = batch["input_ids"].to(CONFIG["device"], non_blocking=True)
        labels = batch["labels"].to(CONFIG["device"], non_blocking=True)
        attention_mask = batch["attention_mask"].to(CONFIG["device"], non_blocking=True)
        user_texts = batch["user_text"]  # 获取用户输入文本

        # 1. 计算SFT损失（标准监督训练损失）
        with torch.autograd.detect_anomaly():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            sft_loss = outputs.loss
            # 梯度累积：Loss除以累积步数
            sft_loss = sft_loss / CONFIG["gradient_accumulation_steps"]

        # 2. 计算对抗损失（ADV Loss）
        adv_loss = torch.tensor(0.0, device=CONFIG["device"], dtype=torch.float32)
        # 动态调整对抗损失权重
        lambda_adv = min(
            CONFIG["lambda_adv_max"],
            CONFIG["lambda_adv_base"] + CONFIG["lambda_adv_step"] * epoch
        )
        
        # 每detector_every_steps计算一次对抗损失
        if global_step % CONFIG["detector_every_steps"] == 0:
            # 随机选择一个样本生成文本
            sample_idx = random.randint(0, len(user_texts) - 1)
            user_text = user_texts[sample_idx]
            
            # 生成文本用于检测器评估
            gen_text = generate_for_detector(model, tokenizer, user_text)
            
            # 计算AI概率作为对抗损失（目标：最小化AI概率）
            ai_prob = detector_ai_prob(gen_text)
            adv_loss = torch.tensor(ai_prob, device=CONFIG["device"], dtype=torch.float32, requires_grad=True)
            # 梯度累积：对抗损失也除以累积步数
            adv_loss = adv_loss / CONFIG["gradient_accumulation_steps"]
            
            # 调试打印（查看ADV Loss详情）
            if global_step % CONFIG["print_every_steps"] == 0:
                print(f"\n📝 ADV Loss 调试 - Step {global_step}:")
                print(f"  用户输入: {user_text[:50]}...")
                print(f"  生成文本: {gen_text[:100]}...")
                print(f"  AI判定概率: {ai_prob:.6f} | 原始ADV Loss: {ai_prob:.6f}")
                print(f"  加权ADV Loss (×{lambda_adv:.3f}): {ai_prob * lambda_adv:.6f}")

        # 3. 计算总损失（SFT Loss + λ * ADV Loss）
        total_loss = sft_loss + lambda_adv * adv_loss

        # 4. 反向传播
        total_loss.backward()

        # 5. 梯度裁剪（防止爆炸）
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.5)

        # 6. 累积步数达标后更新参数
        if (step + 1) % CONFIG["gradient_accumulation_steps"] == 0:
            optimizer.step()
            optimizer.zero_grad()

        # 7. 累计Loss
        epoch_sft_loss += sft_loss.item() * CONFIG["gradient_accumulation_steps"]
        epoch_adv_loss += adv_loss.item() * CONFIG["gradient_accumulation_steps"] * lambda_adv
        avg_sft_loss_list.append(sft_loss.item() * CONFIG["gradient_accumulation_steps"])
        avg_adv_loss_list.append(adv_loss.item() * CONFIG["gradient_accumulation_steps"] * lambda_adv)

        # 8. 打印训练进度
        if global_step % CONFIG["print_every_steps"] == 0:
            step_time = time.time() - step_start
            avg_sft_loss = sum(avg_sft_loss_list[-CONFIG["print_every_steps"]:]) / CONFIG["print_every_steps"]
            avg_adv_loss = sum(avg_adv_loss_list[-CONFIG["print_every_steps"]:]) / CONFIG["print_every_steps"]
            avg_total_loss = avg_sft_loss + avg_adv_loss
            
            print(f"[Epoch {epoch+1}/{CONFIG['num_train_epochs']}] "
                  f"Step {global_step}/{total_steps} | "
                  f"SFT Loss: {avg_sft_loss:.4f} | "
                  f"ADV Loss: {avg_adv_loss:.4f} | "
                  f"Total Loss: {avg_total_loss:.4f} | "
                  f"λ_adv: {lambda_adv:.3f} | "
                  f"Grad Norm: {grad_norm.item():.4f} | "
                  f"Step Time: {step_time:.2f}s")

    # Epoch结束处理
    epoch_time = format_time(time.time() - epoch_start)
    total_train_time += time.time() - epoch_start
    avg_epoch_sft_loss = epoch_sft_loss / len(train_loader)
    avg_epoch_adv_loss = epoch_adv_loss / len(train_loader)
    avg_epoch_total_loss = avg_epoch_sft_loss + avg_epoch_adv_loss
    
    print(f"\n✅ Epoch {epoch+1} 完成 | "
          f"Avg SFT Loss: {avg_epoch_sft_loss:.4f} | "
          f"Avg ADV Loss: {avg_epoch_adv_loss:.4f} | "
          f"Avg Total Loss: {avg_epoch_total_loss:.4f} | "
          f"λ_adv: {lambda_adv:.3f} | "
          f"Epoch Time: {epoch_time} | "
          f"Total Time: {format_time(total_train_time)}")

    # 每个Epoch结束后验证生成效果（包含ADV Loss评估）
    generate_validation_sample(model, tokenizer, dataset, CONFIG)

# -------------------------
# 8. 保存最终模型
# -------------------------
print("\n📦 保存训练完成模型...")
model.save_pretrained(CONFIG["output_dir"])
tokenizer.save_pretrained(CONFIG["output_dir"])
print(f"✅ 模型已保存至: {CONFIG['output_dir']}")

# -------------------------
# 9. 最终生成验证（随机选3条数据，包含ADV Loss评估）
# -------------------------
print("\n🎯 最终生成验证（随机3条数据）:")
for i in range(3):
    generate_validation_sample(model, tokenizer, dataset, CONFIG)

print("\n🎉 全量对抗训练完成！")

/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


✅ 生成模型Tokenizer加载完成

📌 加载AI检测器模型...


Some weights of the model checkpoint at /root/autodl-tmp/Pro/models--roberta-base-openai-detector/snapshots/6cba99c003b711c7fe94f8a3aa2be35a792cb6fa were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✅ AI检测器加载完成并冻结
⚠️  跳过第920行：内容为空
⚠️  跳过第946行：内容为空
⚠️  跳过第4092行：内容为空
⚠️  跳过第4192行：内容为空
⚠️  跳过第4194行：内容为空
⚠️  跳过第4199行：内容为空
⚠️  跳过第4200行：内容为空
⚠️  跳过第4224行：内容为空
⚠️  跳过第4242行：内容为空
⚠️  跳过第4243行：内容为空
⚠️  跳过第4252行：内容为空
⚠️  跳过第4254行：内容为空
⚠️  跳过第4301行：内容为空
⚠️  跳过第4302行：内容为空
⚠️  跳过第4303行：内容为空
⚠️  跳过第4457行：内容为空
⚠️  跳过第4458行：内容为空
⚠️  跳过第4459行：内容为空
⚠️  跳过第4541行：内容为空
⚠️  跳过第4602行：内容为空
⚠️  跳过第4603行：内容为空
⚠️  跳过第4762行：内容为空
⚠️  跳过第4763行：内容为空
⚠️  跳过第5447行：内容为空
⚠️  跳过第5452行：内容为空
⚠️  跳过第5595行：内容为空
⚠️  跳过第5597行：内容为空
⚠️  跳过第5675行：内容为空
⚠️  跳过第5677行：内容为空
⚠️  跳过第5789行：内容为空
⚠️  跳过第5790行：内容为空
⚠️  跳过第5791行：内容为空
⚠️  跳过第5792行：内容为空
⚠️  跳过第5829行：内容为空
⚠️  跳过第5836行：内容为空
⚠️  跳过第5837行：内容为空
⚠️  跳过第5853行：内容为空
⚠️  跳过第5861行：内容为空
⚠️  跳过第5969行：内容为空
⚠️  跳过第5970行：内容为空
⚠️  跳过第5989行：内容为空
⚠️  跳过第5990行：内容为空
⚠️  跳过第6264行：内容为空
⚠️  跳过第6334行：内容为空
⚠️  跳过第6335行：内容为空
⚠️  跳过第6367行：内容为空
⚠️  跳过第6368行：内容为空
⚠️  跳过第6369行：内容为空
⚠️  跳过第6370行：内容为空
⚠️  跳过第6371行：内容为空
⚠️  跳过第6372行：内容为空
⚠️  跳过第6373行：内容为空
⚠️  跳过第6374行：内容为空
⚠️  跳过第6375行：内容为空
⚠️  跳过第6376行：内容

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.45s/it]


trainable params: 1,261,568 || all params: 7,616,878,080 || trainable%: 0.0166

🚀 开始全量数据对抗训练...
[Epoch 1/2] Step 50/13752 | SFT Loss: 6.5800 | ADV Loss: 0.0000 | Total Loss: 6.5800 | λ_adv: 0.020 | Grad Norm: 5.9683 | Step Time: 0.87s


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



📝 ADV Loss 调试 - Step 100:
  用户输入: Write a poem in the style of Judith Goldman....
  生成文本: In twilight's hush, beneath the moon's soft gaze,
The world whispers secrets, tender and true.
A loo...
  AI判定概率: 1.000000 | 原始ADV Loss: 1.000000
  加权ADV Loss (×0.020): 0.020000
[Epoch 1/2] Step 100/13752 | SFT Loss: 5.6021 | ADV Loss: 0.0004 | Total Loss: 5.6025 | λ_adv: 0.020 | Grad Norm: 15.9153 | Step Time: 11.28s
[Epoch 1/2] Step 150/13752 | SFT Loss: 5.4044 | ADV Loss: 0.0000 | Total Loss: 5.4044 | λ_adv: 0.020 | Grad Norm: 7.1493 | Step Time: 0.87s

📝 ADV Loss 调试 - Step 200:
  用户输入: Write a poem using the theme: Living,Time & Brevit...
  生成文本: A river carves its path,
Through valleys and mountainsides.
In time it weaves a thread — 
Of ice or ...
  AI判定概率: 0.998535 | 原始ADV Loss: 0.998535
  加权ADV Loss (×0.020): 0.019971
[Epoch 1/2] Step 200/13752 | SFT Loss: 4.4700 | ADV Loss: 0.0004 | Total Loss: 4.4704 | λ_adv: 0.020 | Grad Norm: 12.2982 | Step Time: 10.90s
[Epoch 1/2] Step 250/13752 | SFT

KeyboardInterrupt: 

In [4]:
#测试ADV Loss可运作
import json
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    AutoModelForSequenceClassification, AutoTokenizer as DetectorTokenizer
)
import warnings
warnings.filterwarnings("ignore")

# ======================================================
# 配置（根据你的实际路径修改）
# ======================================================
JSONL_PATH = "/root/autodl-tmp/Pro/poetry_sft.jsonl"
BASE_MODEL_PATH = "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b"
DETECTOR_MODEL_PATH = "/root/autodl-tmp/Pro/models--roberta-base-openai-detector/snapshots/6cba99c003b711c7fe94f8a3aa2be35a792cb6fa"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
λ_adv = 0.02  # 对抗损失权重（和你的训练日志一致）
MAX_NEW_TOKENS = 128
DETECTOR_DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32  # 统一检测器 dtype

# ======================================================
# 1. 仅加载第一条JSONL样本
# ======================================================
def load_single_sample(jsonl_path):
    """加载JSONL文件的第一条样本"""
    with open(jsonl_path, "r", encoding="utf-8") as f:
        first_line = next(f)
        sample = json.loads(first_line.strip())
    
    # 构造训练用的prompt（和你的训练格式一致）
    user_msg = sample["messages"][0]["content"]
    assistant_msg = sample["messages"][1]["content"]
    
    prompt = (
        f"<|im_start|>user\n{user_msg}<|im_end|>\n"
        f"<|im_start|>assistant\n{assistant_msg}<|im_end|>"
    )
    return {
        "prompt": prompt,
        "user_msg": user_msg,
        "assistant_msg": assistant_msg,
        "raw_sample": sample
    }

# ======================================================
# 2. 加载模型和检测器（修复dtype + 真实检测器）
# ======================================================
def load_models():
    """加载基础模型和真实的AI检测器（roberta-base-openai-detector）"""
    # --------------------------
    # 1. 加载生成模型（Qwen2.5-7B）
    # --------------------------
    tokenizer = AutoTokenizer.from_pretrained(
        BASE_MODEL_PATH,
        trust_remote_code=True,
        padding_side="right"
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_PATH,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    model.eval()  # 仅前向计算，不训练
    
    # --------------------------
    # 2. 加载真实的AI检测器（roberta-base-openai-detector）
    # --------------------------
    # 加载检测器tokenizer
    detector_tokenizer = DetectorTokenizer.from_pretrained(
        DETECTOR_MODEL_PATH,
        padding_side="right",
        trust_remote_code=True
    )
    
    # 加载检测器模型（修复dtype问题，统一为float16/float32）
    detector = AutoModelForSequenceClassification.from_pretrained(
        DETECTOR_MODEL_PATH,
        torch_dtype=DETECTOR_DTYPE,
        device_map="auto"
    )
    detector.eval()  # 冻结检测器，仅前向
    
    return tokenizer, model, detector_tokenizer, detector

# ======================================================
# 3. 核心：计算单样本的SFT Loss和ADV Loss
# ======================================================
def compute_losses(gen_tokenizer, gen_model, det_tokenizer, detector, sample):
    """计算单样本的SFT Loss和ADV Loss"""
    # --------------------------
    # 步骤1：Tokenize输入（生成模型）
    # --------------------------
    prompt = sample["prompt"]
    inputs = gen_tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024
    ).to(DEVICE)
    
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    
    # --------------------------
    # 步骤2：计算SFT Loss（标准语言模型损失）
    # --------------------------
    with torch.no_grad():
        outputs = gen_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=input_ids  # 自回归标签
        )
    sft_loss = outputs.loss.item()
    
    # --------------------------
    # 步骤3：生成文本（用于计算ADV Loss）
    # --------------------------
    generate_inputs = gen_tokenizer(
        f"<|im_start|>user\n{sample['user_msg']}<|im_end|>\n<|im_start|>assistant\n",
        return_tensors="pt",
        padding=True
    ).to(DEVICE)
    
    with torch.no_grad():
        gen_ids = gen_model.generate(
            **generate_inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=0.0,
            do_sample=False,
            pad_token_id=gen_tokenizer.pad_token_id,
            eos_token_id=gen_tokenizer.eos_token_id
        )
    
    # 解码生成的文本
    gen_text = gen_tokenizer.decode(
        gen_ids[0][generate_inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    ).strip()
    
    # --------------------------
    # 步骤4：计算ADV Loss（核心调试部分）
    # --------------------------
    print("\n=== ADV Loss 调试信息 ===")
    print(f"生成文本: {gen_text[:200]}...")
    
    # --------------------------
    # 4.1 Tokenize生成文本（检测器）
    # --------------------------
    det_inputs = det_tokenizer(
        gen_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(DEVICE)
    
    # --------------------------
    # 4.2 检测器前向计算（修复dtype问题）
    # --------------------------
    with torch.no_grad():
        det_outputs = detector(**det_inputs)
        # roberta-base-openai-detector输出logits：[0]=人类概率，[1]=AI概率
        logits = det_outputs.logits
        ai_prob = torch.softmax(logits, dim=-1)[0][1].item()  # AI生成的概率（0~1）
    
    print(f"检测器AI判定概率: {ai_prob:.6f}")
    
    # --------------------------
    # 4.3 计算对抗损失（最小化AI判定概率）
    # --------------------------
    raw_adv_loss = ai_prob  # 原始对抗损失（AI概率值）
    weighted_adv_loss = raw_adv_loss * λ_adv  # 加权后对抗损失
    
    print(f"原始ADV Loss (AI概率): {raw_adv_loss:.6f}")
    print(f"加权后ADV Loss (×λ_adv={λ_adv}): {weighted_adv_loss:.6f}")
    
    # --------------------------
    # 返回结果
    # --------------------------
    return {
        "sft_loss": sft_loss,
        "raw_adv_loss": raw_adv_loss,
        "weighted_adv_loss": weighted_adv_loss,
        "gen_text": gen_text,
        "detector_ai_prob": ai_prob
    }

# ======================================================
# 4. 主函数：运行单样本调试
# ======================================================
if __name__ == "__main__":
    print("===== 单样本 ADV Loss 调试 =====")
    
    # 加载单样本
    print("\n1. 加载第一条JSONL样本...")
    sample = load_single_sample(JSONL_PATH)
    print(f"样本User内容: {sample['user_msg'][:100]}...")
    print(f"样本Assistant内容: {sample['assistant_msg'][:100]}...")
    
    # 加载模型（生成模型 + 真实检测器）
    print("\n2. 加载模型和检测器...")
    gen_tokenizer, gen_model, det_tokenizer, detector = load_models()
    print("模型和检测器加载完成！")
    
    # 计算损失
    print("\n3. 计算SFT Loss和ADV Loss...")
    loss_results = compute_losses(gen_tokenizer, gen_model, det_tokenizer, detector, sample)
    
    # 打印最终结果
    print("\n===== 最终结果 =====")
    print(f"SFT Loss: {loss_results['sft_loss']:.6f}")
    print(f"原始ADV Loss (AI概率): {loss_results['raw_adv_loss']:.6f}")
    print(f"加权后ADV Loss: {loss_results['weighted_adv_loss']:.6f}")
    print(f"检测器判定为AI的概率: {loss_results['detector_ai_prob']:.6f}")
    
    # 验证是否为0
    if abs(loss_results["raw_adv_loss"]) < 1e-6:
        print("\n❌ 异常：ADV Loss 接近 0！")
    else:
        print("\n✅ 正常：ADV Loss 非 0！")

===== 单样本 ADV Loss 调试 =====

1. 加载第一条JSONL样本...
样本User内容: Write a poem in the style of Michelle Menting....
样本Assistant内容: Dog bone, stapler,
cribbage board, garlic press
     because this window is loose—lacks
suction, lac...

2. 加载模型和检测器...


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.21it/s]
Some weights of the model checkpoint at /root/autodl-tmp/Pro/models--roberta-base-openai-detector/snapshots/6cba99c003b711c7fe94f8a3aa2be35a792cb6fa were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


模型和检测器加载完成！

3. 计算SFT Loss和ADV Loss...

=== ADV Loss 调试信息 ===
生成文本: In the quiet of dawn's first light,
Where whispers of mist gently take flight,
A world awakens, soft and serene,
In hues of gold and shades of green.

The sun peeks through the sleepy clouds,
Its warm...
检测器AI判定概率: 0.999512
原始ADV Loss (AI概率): 0.999512
加权后ADV Loss (×λ_adv=0.02): 0.019990

===== 最终结果 =====
SFT Loss: 3.845113
原始ADV Loss (AI概率): 0.999512
加权后ADV Loss: 0.019990
检测器判定为AI的概率: 0.999512

✅ 正常：ADV Loss 非 0！
